In [1]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# %cd /content/gdrive/MyDrive/DATASETS/CSE-CIC-IDS-2018_KAGGLE/CSE_CIC_IDS_ALL

In [2]:
%cd /content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/

/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018


In [ ]:
!ls -lrt

total 3288330
-rw------- 1 root root    104482 Feb  5  2024 'SQL Injection.csv'
-rw------- 1 root root  14111086 Feb  5  2024 'DoS attacks-Slowloris.csv'
-rw------- 1 root root   2093031 Feb  5  2024 'DDOS attack-LOIC-UDP.csv'
-rw------- 1 root root    273431 Feb  5  2024 'Brute Force -XSS.csv'
-rw------- 1 root root    770429 Feb  5  2024 'Brute Force -Web.csv'
-rw------- 1 root root  52375241 Feb  5  2024 'DoS attacks-GoldenEye.csv'
-rw------- 1 root root 155416507 Feb  5  2024 'DoS attacks-SlowHTTPTest.csv'
-rw------- 1 root root 365072331 Feb  5  2024  Bot.csv
-rw------- 1 root root 804021836 Feb  5  2024 'DDOS attack-HOIC.csv'
-rw------- 1 root root 194111176 Feb  5  2024  Infilteration.csv
-rw------- 1 root root 218035465 Feb  5  2024  FTP-BruteForce.csv
-rw------- 1 root root 526095460 Feb  5  2024 'DoS attacks-Hulk.csv'
-rw------- 1 root root 803522632 Feb  5  2024 'DDoS attacks-LOIC-HTTP.csv'
-rw------- 1 root root 231234571 Feb  5  2024  SSH-Bruteforce.csv
drwx------ 2 root r

In [3]:
# -*- coding: utf-8 -*-
# Bilevel IDS training (inner BCE on normal-logit, outer ODIN BCE) – checklist-complete edition.

import os, gc, glob, math, random
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import os
import matplotlib
matplotlib.use("Agg")  # force non-GUI backend so nothing tries to display

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import (
    f1_score, precision_score, recall_score, accuracy_score,
    average_precision_score, precision_recall_curve, auc, roc_auc_score
)

In [4]:
import os
from typing import Dict, Tuple
import re

# ---------- robust path resolve (fix common hyphen/underscore mixups) ----------
def resolve_data_dir(path: str) -> str:
    cand = os.path.expanduser(path.rstrip("/ "))
    if os.path.isdir(cand):
        return cand
    # try common variants
    variants = [
        cand.replace("CSE-CIC-IDS_ALL_DATA", "CSE_CIC_IDS_ALL_DATA"),
        cand.replace("CSE_CIC_IDS_ALL_DATA", "CSE-CIC-IDS_ALL_DATA"),
    ]
    for v in variants:
        if os.path.isdir(v):
            return v
    raise FileNotFoundError(f"Directory not found: {path}\nTried: {cand}\n"
                            f"Hint: check underscores vs hyphens in 'CSE_CIC_IDS_ALL_DATA'.")

# ---------- filename cleaning for category/subcategory ----------
def _clean_display_name(name: str) -> str:
    s = name.strip()
    if len(s) >= 2 and s[0] == s[-1] and s[0] in ("'", '"'):
        s = s[1:-1]
    return s

def _parse_cat_subcat_from_display_name(clean_name: str) -> Tuple[str, str]:
    parts = re.split(r"[-–—_]", clean_name, maxsplit=1)
    if len(parts) == 2:
        return parts[0].strip(), parts[1].strip()
    return clean_name.strip(), ""

def discover_files_by_category(data_dir: str) -> Dict[str, str]:
    """
    Finds CSVs case-insensitively, robust to spaces/quotes in names.
    Key is "Category::Subcategory" (or just "Category" if none).
    """
    data_dir = resolve_data_dir(data_dir)
    out: Dict[str, str] = {}
    with os.scandir(data_dir) as it:
        for entry in it:
            if entry.is_file() and entry.name.lower().endswith(".csv"):
                cleaned = _clean_display_name(os.path.splitext(entry.name)[0])
                cat, sub = _parse_cat_subcat_from_display_name(cleaned)
                fam_key = f"{cat}::{sub}" if sub else cat
                out[fam_key] = entry.path  # exact filesystem path
    return out

# ---------- USE IT ----------
DATA_DIR = "/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/"
DATA_DIR = resolve_data_dir(DATA_DIR)  # will raise with a helpful message if wrong

files = discover_files_by_category(DATA_DIR)
if not files:
    raise RuntimeError(f"No CSVs found in {DATA_DIR}")

print("Discovered families:")
for k, v in sorted(files.items()):
    print(f"  {k} -> {repr(v)}")


Discovered families:
  Bot -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/Bot.csv'
  Brute Force::Web -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/Brute Force -Web.csv'
  Brute Force::XSS -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/Brute Force -XSS.csv'
  DDOS attack::HOIC -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/DDOS attack-HOIC.csv'
  DDOS attack::LOIC-UDP -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/DDOS attack-LOIC-UDP.csv'
  DDoS attacks::LOIC-HTTP -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/DDoS attacks-LOIC-HTTP.csv'
  DoS attacks::GoldenEye -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/DoS attacks-GoldenEye.csv'
  DoS attacks::Hulk -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Pr

In [5]:


# ------------------------ Config ------------------------
#DATA_DIR = "/content/gdrive/MyDrive/DATASETS/CSE-CIC-IDS-2018_KAGGLE/CSE_CIC_IDS_ALL/processed"

DATA_DIR = "/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/"

# Episodes
N_EPISODES   = 20
INNER_EPOCHS = 10
OUTER_EPOCHS = 5

INNER_MAX_FAMS = 2
OUTER_MAX_FAMS = 2         # <-- allow 2 OOD families / batch

INNER_POS_PER_FAM          = 20000
INNER_NEG_PER_FAM          = "match"
OUTER_OOD_POS_PER_FAM      = 2000
OUTER_ID_NEG_PER_INNER_FAM = 20000

INNER_BATCH = 256
OUTER_BATCH = 128

INNER_EPISODE_MAX = None
OUTER_EPISODE_MAX = None

#HELDOUT_FAMILIES: List[str] = ["Web1", "DDoS1", "Infil2", "DoS2", "Bruteforce"]

#HELDOUT_FAMILIES: List[str] = ['Brute Force::Web', 'Brute Force::XSS', 'DDOS attack::HOIC', "Infilteration"]

import os, re, unicodedata, difflib


def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKC", s or "")
    s = s.replace("\u00A0", " ")
    s = re.sub(r"[‐-‒–—−]+", "-", s)
    s = re.sub(r"\s+", " ", s).strip().lower()
    return s

def resolve_heldout(user_keys, discovered_keys):
    # normalize discovered keys -> original
    norm2orig = {_norm(k): k for k in discovered_keys}
    resolved, missing = [], []
    for k in user_keys:
        nk = _norm(k)
        if nk in norm2orig:
            resolved.append(norm2orig[nk])
        else:
            # suggest close matches
            cand = difflib.get_close_matches(nk, list(norm2orig.keys()), n=3, cutoff=0.6)
            missing.append((k, [norm2orig[c] for c in cand]))
    return resolved, missing


# ---------------- Example wiring ----------------

# after you’ve discovered files (keys) with your discover_files_by_category():
files = discover_files_by_category(DATA_DIR)   # dict: {family_key: path}
discovered = sorted(files.keys())
print("Discovered families:")
for k in discovered:
    print("  ", k)

# your intended held-out list (as you like to type it):
HELDOUT_FAMILIES = [
    'Brute Force::Web',
    'Brute Force::XSS',
    'DDOS attack::HOIC',
    'Infilteration',
    'DoS attacks::GoldenEye',
    'FTP::BruteForce',
    'DoS attacks::Slowloris'


]

# resolve to the exact discovered keys
heldout_resolved, missing = resolve_heldout(HELDOUT_FAMILIES, discovered)

if missing:
    print("\nCould not resolve some HELDOUT_FAMILIES exactly:")
    for orig, hints in missing:
        if hints:
            print(f"  - '{orig}'  -> did you mean: {hints}")
        else:
            print(f"  - '{orig}'  -> no close match found")
    # you can choose to raise here if you want:
    # raise ValueError("Fix HELDOUT_FAMILIES entries above.")

# use the resolved names going forward
heldout_fams = heldout_resolved
seen_train_fams = [f for f in discovered if f not in heldout_fams]

print("\nHeld-out (eval-only):", heldout_fams)
print("Seen train families :", seen_train_fams)


VAL_PER_CLASS_CAP  = 25_000
TEST_PER_CLASS_CAP = 25_000

ACCUM_STEPS     = 1
USE_AMP         = True
CLIP_NORM       = 1.0
WEIGHT_DECAY    = 1e-4
LABEL_SMOOTH    = 0.02

NUM_WORKERS     = 0
PIN_MEMORY      = False
INNER_LR_MAX    = 3e-3

# Optional reconstruction weight
LAMBDA_REC = 0.10

# Model dims
EMBED_DIM  = 128
ENC_HIDDEN = 256
ENC_DEPTH  = 3
DEC_HIDDEN = 128
DEC_DEPTH  = 2
DROPOUT    = 0.10

# ODIN config (learn both T and eps)
TEMP_INIT       = 1.0
EPS_INIT        = 2e-3
TUNE_TEMPERATURE= True
TUNE_EPSILON    = True

# ODIN step control
TRAIN_PGD_STEPS = 5
VAL_PGD_STEPS   = 1      # speed up validation
ODIN_TAU        = 1e-2   # EMA pullback

# Clamp ranges
T_MIN,  T_MAX    = 0.3, 1.05
EPS_MIN, EPS_MAX = 5e-4, 3e-2

# Robust-threshold building
EASY_AP_CUTOFF = 0.98   # families with AP >= this are "easy" and excluded from robust median


# ----- Calibration controls -----
CALIB_USE_MIN_PREC = True   # if True, pick per-family thr at F1* subject to Precision >= CALIB_MIN_PREC
CALIB_MIN_PREC     = 0.80   # minimum precision constraint for calibration
CALIB_THR_FLOOR    = 1e-3   # never allow global thr below this floor


# ----- Outer margin ("hard separation") -----
OUTER_MARGIN_M     = 0.8    # desired gap: logit_norm(ID) - logit_norm(OOD) >= M
OUTER_MARGIN_ALPHA = 0.4   # strength of the margin penalty


# Prioritize tough seen families for outer OOD (held-out families never appear here)
# HARD_OOD_FAMS = ["Infil1", "DoS1", "Web2"]  # order = priority; will auto-filter to allowed
# TARGET_OOD_K   = 3                           # try to keep 2–3 OOD families (capped by OUTER_MAX_FAMS)

# OUTER_MAX_FAMS = 3            # was likely 2
# HARD_OOD_FAMS = ["Infil1", "Infil2" , "DoS1", "Web2"]  # order is a seed; we’ll rotate it
# TARGET_OOD_K  = 3
# _rr_ptr = 0                   # module-level or kept in your runner's state

# ===== Few-shot OUTER knobs (NEW) =====
FS_NUM_OOD_CLASSES     = 4    # how many OOD families per outer step
FS_K_PER_CLASS_OUTER   = 100   # anomalies per fam, and equal number of normals from same fam
FS_ALLOW_SMALLER       = True # if a fam lacks K, take what’s available

# (kept) “hard” families preference for choosing candidates
OUTER_MAX_FAMS = 3
TARGET_OOD_K   = 3
#HARD_OOD_FAMS  = ["Infil1", "Infil2", "DoS1", "Web1", "Web2"]
HARD_OOD_FAMS= []
rr_ptr = 0

Discovered families:
   Bot
   Brute Force::Web
   Brute Force::XSS
   DDOS attack::HOIC
   DDOS attack::LOIC-UDP
   DDoS attacks::LOIC-HTTP
   DoS attacks::GoldenEye
   DoS attacks::Hulk
   DoS attacks::SlowHTTPTest
   DoS attacks::Slowloris
   FTP::BruteForce
   Infilteration
   SQL Injection
   SSH::Bruteforce

Held-out (eval-only): ['Brute Force::Web', 'Brute Force::XSS', 'DDOS attack::HOIC', 'Infilteration', 'DoS attacks::GoldenEye', 'FTP::BruteForce', 'DoS attacks::Slowloris']
Seen train families : ['Bot', 'DDOS attack::LOIC-UDP', 'DDoS attacks::LOIC-HTTP', 'DoS attacks::Hulk', 'DoS attacks::SlowHTTPTest', 'SQL Injection', 'SSH::Bruteforce']


In [6]:



# # ===== Few-shot OOD controls =====
# FS_NUM_OOD_CLASSES =

# FS_K_PER_CLASS_OUTER = 500


# ------------------------ Globals ------------------------
SEED = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed: int = SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

def discover_files(data_dir: str) -> Dict[str, str]:
    paths = sorted(glob.glob(os.path.join(data_dir, "*.csv")))
    out = {}
    for p in paths:
        if os.path.isdir(p):  # guard
            continue
        name = os.path.basename(p)
        fam  = name.split("-")[0]
        out[fam] = p
    return out

def _find_label_column(df: pd.DataFrame) -> str:
    for cand in ["Label", "label", "labels", "y", "target", "class", "Class"]:
        if cand in df.columns: return cand
    last = df.columns[-1]
    if set(pd.unique(df[last]).tolist()).issubset({0,1}): return last
    raise RuntimeError("Could not find binary label column (0/1).")




def safe_savefig(fig, out_path: str):
    """Create parent dirs, save fig, and close it. Robust to missing dirs."""
    import os, matplotlib.pyplot as plt
    out_dir = os.path.dirname(out_path)
    if out_dir:
        os.makedirs(out_dir, exist_ok=True)
    # save and close (no bbox_inches="tight" here!)
    fig.savefig(out_path, dpi=150, bbox_inches=None, pad_inches=0.05)
    plt.close(fig)

In [7]:
import os, re
from typing import Dict, Tuple

def _clean_display_name(name: str) -> str:
    """
    Clean only for building the family key (NOT for path):
      - strip leading/trailing whitespace
      - strip wrapping single/double quotes if present
    """
    s = name.strip()
    if (len(s) >= 2) and ((s[0] == s[-1]) and s[0] in ("'", '"')):
        s = s[1:-1]
    return s

def _parse_cat_subcat_from_display_name(clean_name: str) -> Tuple[str, str]:
    """
    From a cleaned base name (without extension), extract (category, subcategory).
    Split on the FIRST '-', '–', '—', or '_' only.
    """
    parts = re.split(r"[-–—_]", clean_name, maxsplit=1)
    if len(parts) == 2:
        cat = parts[0].strip()
        sub = parts[1].strip()
    else:
        cat, sub = clean_name.strip(), ""
    return cat, sub

def discover_files_by_category(data_dir: str) -> Dict[str, str]:
    """
    Use os.scandir to robustly find CSVs (case-insensitive), regardless of spaces/quotes.
    Returns: { fam_key: full_path }
    fam_key = "Category::Subcategory" or "Category" if no subcategory.
    """
    out: Dict[str, str] = {}
    with os.scandir(data_dir) as it:
        for entry in it:
            if not entry.is_file():
                continue
            # Case-insensitive .csv check (does not care about surrounding quotes)
            if not entry.name.lower().endswith(".csv"):
                continue

            # Build fam key from a cleaned *display* name, not the path
            cleaned = _clean_display_name(os.path.splitext(entry.name)[0])
            cat, sub = _parse_cat_subcat_from_display_name(cleaned)
            fam_key = f"{cat}::{sub}" if sub else cat

            # Keep the true filesystem path exactly as-is for reading
            out[fam_key] = entry.path
    return out


In [8]:
import numpy as np
import pandas as pd
from typing import Dict, List, Optional
from sklearn.preprocessing import StandardScaler, RobustScaler

def _find_label_column(df: pd.DataFrame) -> str:
    for cand in ["Label", "label", "labels", "y", "target", "class", "Class"]:
        if cand in df.columns:
            return cand
    # last-column fallback if it looks binary
    last = df.columns[-1]
    if set(pd.unique(df[last]).tolist()).issubset({0, 1}):
        return last
    raise RuntimeError("Could not find binary label column (0/1).")

def _split_fam_key(fam_key: str) -> Tuple[str, str]:
    # fam_key is either "Category::Subcategory" or "Category"
    if "::" in fam_key:
        a, b = fam_key.split("::", 1)
        return a.strip(), b.strip()
    return fam_key.strip(), ""

class DataCache:
    def __init__(self, files: Dict[str, str], max_rows: Optional[int] = None):
        self.files = files                      # fam_key -> path
        self.max_rows = max_rows
        self.X: Dict[str, np.ndarray] = {}      # fam_key -> X
        self.y: Dict[str, np.ndarray] = {}      # fam_key -> y
        self.splits = None
        self.scaler = None

        # --- NEW: metadata (does not affect any existing behavior) ---
        self.meta: Dict[str, Dict[str, str]] = {}
        for fam in files.keys():
            cat, sub = _split_fam_key(fam)
            self.meta[fam] = {"category": cat, "subcategory": sub}

    def load_all(self):
        for fam, path in self.files.items():
            df = pd.read_csv(path, nrows=None if self.max_rows is None else int(self.max_rows))
            y_col = _find_label_column(df)
            df.replace([np.inf, -np.inf], np.nan, inplace=True)
            df.dropna(axis=0, how="any", inplace=True)
            df = df[df[y_col].isin([0, 1])]
            y = df[y_col].astype(np.int32).to_numpy()
            X = df.drop(columns=[y_col]).astype(np.float32).to_numpy()
            finite_mask = np.isfinite(X).all(axis=1)
            X, y = X[finite_mask], y[finite_mask]
            self.X[fam] = X
            self.y[fam] = y

    def build_splits(self, rng: np.random.Generator, train_frac=2/3, val_frac=1/6, test_frac=1/6):
        splits = {}
        for fam in self.files.keys():
            y = self.y[fam]
            famspl = {0: {}, 1: {}}
            for c in (0, 1):
                idx = np.where(y == c)[0].copy()
                rng.shuffle(idx)
                n = idx.size
                n_tr = int(round(n * train_frac))
                n_va = int(round(n * val_frac))
                famspl[c]['train'] = idx[:n_tr]
                famspl[c]['val']   = idx[n_tr:n_tr+n_va]
                famspl[c]['test']  = idx[n_tr+n_va:]
            splits[fam] = famspl
        self.splits = splits

    def count(self, fam: str, cls: int, part: str) -> int:
        return int(self.splits[fam][cls][part].size)

    def families_with(self, cls: int, part: str) -> List[str]:
        return [f for f in self.files.keys() if self.count(f, cls, part) > 0]

    def sample(self, fam: str, cls: int, part: str, k: int, rng: np.random.Generator) -> np.ndarray:
        idx = self.splits[fam][cls][part]
        if idx.size == 0:
            D = self.X[next(iter(self.X))].shape[1]
            return np.empty((0, D), np.float32)
        sel = idx if k >= idx.size else rng.choice(idx, size=int(k), replace=False)
        return self.X[fam][sel].astype(np.float32, copy=False)

    def fit_scaler_on_train(self, rng, use_robust=True):
        fams = list(self.files.keys())
        X_train_chunks = []
        for fam in fams:
            idx_pos = self.splits[fam][1]['train']
            idx_neg = self.splits[fam][0]['train']
            for idx in (idx_pos, idx_neg):
                if idx.size == 0:
                    continue
                take = min(10000, idx.size)
                sel = idx if idx.size <= take else rng.choice(idx, take, replace=False)
                X_train_chunks.append(self.X[fam][sel])
        if not X_train_chunks:
            self.scaler = None
            return
        X_fit = np.vstack(X_train_chunks)
        self.scaler = RobustScaler(quantile_range=(5, 95)) if use_robust else StandardScaler()
        self.scaler.fit(X_fit)

    def apply_scaler_all(self):
        if self.scaler is None:
            return
        for fam in self.files.keys():
            self.X[fam] = self.scaler.transform(self.X[fam]).astype(np.float32, copy=False)

    # ---- convenience (optional) ----
    def get_categories(self) -> Dict[str, List[str]]:
        """
        category -> list of subcategories ('' if none).
        """
        out: Dict[str, List[str]] = {}
        for fam, m in self.meta.items():
            out.setdefault(m["category"], []).append(m["subcategory"])
        return out


In [9]:

    # ---- SNAPSHOT PANELS (reuse your plot_id_ood_separation) ----
def snapshot_sep(cache, encoder, clf, odin, rng, episode: int, label: str,
                part="val", cap_per_family=4000, bins=60):
      """
      Saves 3-panel (hist/ROC/PR) for the given checkpoint label.
      Files go to: plots/ep{episode:02d}_{label}/sep_latest.png
      """
      import os
      out_dir = os.path.join("plots", f"ep{episode:02d}_{label}")
      os.makedirs(out_dir, exist_ok=True)
      return plot_id_ood_separation(
          cache, encoder, clf, odin, rng=rng,
          part=part, families=None, cap_per_family=cap_per_family,
          bins=bins, episode=None, save_dir=out_dir, show=False
      )

In [10]:
    # ---- TRAJECTORY RECORDER ----
class StageTrajectory:
      def __init__(self, episode: int, save_dir="plots"):
          self.episode = episode
          self.save_dir = save_dir
          self.inner_auc, self.inner_ap = [], []
          self.outer_auc, self.outer_ap = [], []

      def eval_and_push(self, cache, encoder, clf, odin, rng, stage: str):
          res = plot_id_ood_separation(
              cache, encoder, clf, odin, rng=rng,
              part="val", cap_per_family=2000, bins=60,
              episode=None, save_dir=None, show=False
          )
          if stage == "inner":
              self.inner_auc.append(res["roc_auc"]); self.inner_ap.append(res["ap"])
          else:
              self.outer_auc.append(res["roc_auc"]); self.outer_ap.append(res["ap"])

      def save_plot(self):
          import os, matplotlib.pyplot as plt
          os.makedirs(self.save_dir, exist_ok=True)
          fig = plt.figure(figsize=(9, 6))
          ax = fig.add_subplot(2,1,1)
          ax.plot(self.inner_auc + self.outer_auc, lw=2)
          ax.axvline(len(self.inner_auc)-1, ls="--", lw=1)
          ax.set_title(f"Episode {self.episode:02d} • AUC-ROC trajectory")
          ax.set_xlabel("Step"); ax.set_ylabel("AUC-ROC")
          ax2 = fig.add_subplot(2,1,2)
          ax2.plot(self.inner_ap + self.outer_ap, lw=2)
          ax2.axvline(len(self.inner_ap)-1, ls="--", lw=1)
          ax2.set_title("AP trajectory"); ax2.set_xlabel("Step"); ax2.set_ylabel("AP")
          fig.tight_layout()
          fig.savefig(os.path.join(self.save_dir, f"ep{self.episode:02d}_trajectory.png"),
                      dpi=150, bbox_inches="tight")
          plt.close(fig)

In [11]:

# ---- ODIN TELEMETRY ----
class OdinTelemetry:
    def __init__(self, episode: int, save_dir="plots"):
        self.episode = episode
        self.save_dir = save_dir
        self.rows = []  # dicts with step, T, eps, g_logT, g_logeps, loss

    def push(self, step, odin, g_logT=None, g_logeps=None, loss=None, device="cpu"):
        self.rows.append({
            "step": int(step),
            "T": float(odin.T.detach().to("cpu")),
            "eps": float(odin.eps.detach().to("cpu")),
            "g_logT": float(0.0 if g_logT is None else g_logT),
            "g_logeps": float(0.0 if g_logeps is None else g_logeps),
            "loss": float("nan" if loss is None else float(loss))
        })

    def save_csv_and_plot(self):
        import os, pandas as pd, matplotlib.pyplot as plt
        os.makedirs(self.save_dir, exist_ok=True)
        path = os.path.join(self.save_dir, f"ep{self.episode:02d}_odin_telemetry.csv")
        df = pd.DataFrame(self.rows)
        df.to_csv(path, index=False)

        fig, ax = plt.subplots(2,1, figsize=(9,6), sharex=True)
        ax[0].plot(df["step"], df["T"], lw=2, label="T")
        ax[0].plot(df["step"], df["eps"], lw=2, label="epsilon")
        ax[0].set_ylabel("Value"); ax[0].legend()

        ax[1].plot(df["step"], df["g_logT"], lw=1.5, label="||∇logT||")
        ax[1].plot(df["step"], df["g_logeps"], lw=1.5, label="||∇logε||")
        ax[1].set_xlabel("Outer step"); ax[1].set_ylabel("Grad norm"); ax[1].legend()

        fig.tight_layout()
        fig.savefig(os.path.join(self.save_dir, f"ep{self.episode:02d}_odin_telemetry.png"),
                    dpi=150, bbox_inches="tight")
        plt.close(fig)

In [12]:
  # ---- FAMILY MARGIN ----
def compute_family_margin(cache, encoder, clf, odin, rng, part="val", cap=2000):
    import numpy as np, torch
    fams = sorted(cache.files.keys())
    out = {}
    for fam in fams:
        m = min(cache.count(fam,1,part), cache.count(fam,0,part), cap)
        if m <= 0:
            continue
        Xa = cache.sample(fam, 1, part, m, rng).astype(np.float32)
        Xn = cache.sample(fam, 0, part, m, rng).astype(np.float32)
        with torch.no_grad():
            Sa = odin_normal_logit(torch.from_numpy(Xa).to(device), encoder, clf, odin,
                                   steps=VAL_PGD_STEPS).detach().cpu().numpy()
            Sn = odin_normal_logit(torch.from_numpy(Xn).to(device), encoder, clf, odin,
                                   steps=VAL_PGD_STEPS).detach().cpu().numpy()
        out[fam] = float(Sn.mean() - Sa.mean())
    return out

def plot_family_margin(m_pre: dict, m_inner: dict, m_outer: dict,
                       episode: int, m_target: float, save_dir="plots"):
    import os, matplotlib.pyplot as plt, numpy as np
    os.makedirs(save_dir, exist_ok=True)
    fams = sorted(set(m_pre) | set(m_inner) | set(m_outer))
    x = np.arange(len(fams)); w = 0.25
    fig, ax = plt.subplots(figsize=(max(10, len(fams)*0.6), 4.5))
    ax.bar(x - w, [m_pre.get(f, np.nan) for f in fams], width=w, label="pre")
    ax.bar(x      , [m_inner.get(f, np.nan) for f in fams], width=w, label="after_inner")
    ax.bar(x + w, [m_outer.get(f, np.nan) for f in fams], width=w, label="after_outer")
    ax.axhline(m_target, ls="--", lw=1, color="k", label="margin m")
    ax.set_xticks(x); ax.set_xticklabels(fams, rotation=45, ha="right")
    ax.set_ylabel("Mean calibrated logit gap (ID − OOD)")
    ax.set_title(f"Episode {episode:02d} • per-family margin")
    ax.legend()
    fig.tight_layout()
    fig.savefig(os.path.join(save_dir, f"ep{episode:02d}_family_margin.png"),
                dpi=150, bbox_inches="tight")
    plt.close(fig)

In [13]:
# ============================ helpers: per-family eval ============================

def build_perfam_eval_pools(cache, fams, rng, part: str, cap_per_family: int):
    """Return {fam: (X, y)} with balanced ID(0)/OOD(1) blocks per family from split `part`."""
    def _balanced_block(fam):
        m = min(cache.count(fam, 1, part), cache.count(fam, 0, part), cap_per_family)
        if m <= 0:
            D = cache.X[fam].shape[1]
            return np.empty((0, D), np.float32), np.empty((0,), np.int64)
        Xa = cache.sample(fam, 1, part, m, rng)  # OOD(+1)
        Xn = cache.sample(fam, 0, part, m, rng)  # ID(0)
        X  = np.vstack([Xa, Xn]).astype(np.float32, copy=False)
        y  = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)])
        return X, y

    pools = {}
    for fam in fams:
        Xf, yf = _balanced_block(fam)
        if Xf.size > 0:
            pools[fam] = (Xf, yf)
    return pools


def score_anomaly_numpy(X, encoder, clf, odin, steps: int = 1):
    """Anomaly score = 1 - p_norm via your ODIN routine (no tau argument)."""
    p_norm = odin_pnorm_numpy(X, encoder, clf, odin, steps=steps)
    return 1.0 - p_norm


def eval_stage_per_family(eval_pools, encoder, clf, odin, steps: int, stage_name: str):
    """
    Compute per-family anomaly scores for a given stage.
    Returns: { fam: {'y': y, 'scores': s} }
    """
    out = {}
    for fam, (Xf, yf) in eval_pools.items():
        s = score_anomaly_numpy(Xf, encoder, clf, odin, steps=steps)
        out[fam] = {"y": yf, "scores": s, "stage": stage_name}
    return out


def plot_per_family_overlays(
    results_pre,
    results_inner,
    results_outer,
    episode: int,
    save_root: str = "plots_perfam",
    bins: int = 60,
    verbose: bool = False,        # <— add back
):
    import os, matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc
    import matplotlib.ticker as mticker

    os.makedirs(save_root, exist_ok=True)
    save_dir = os.path.join(save_root, f"ep{episode:02d}")
    os.makedirs(save_dir, exist_ok=True)

    fams = sorted(results_pre.keys())
    saved = []

    for fam in fams:
        y  = results_pre[fam]["y"]
        s0 = results_pre[fam]["scores"]
        s1 = results_inner.get(fam, {}).get("scores", None)
        s2 = results_outer.get(fam, {}).get("scores", None)

        # narrow: squeeze x by reducing figure width
        fig = plt.figure(figsize=(4.5, 6.0))

        # (1) ID hist
        ax1 = plt.subplot(2, 2, 1)
        ax1.hist(s0[y==0], bins=bins, alpha=0.35, density=True, label="ID pre",   color="tab:blue")
        if s1 is not None: ax1.hist(s1[y==0], bins=bins, alpha=0.35, density=True, label="ID inner", color="tab:orange")
        if s2 is not None: ax1.hist(s2[y==0], bins=bins, alpha=0.35, density=True, label="ID outer", color="tab:green")
        ax1.set_title(f"{fam} — ID hist"); ax1.set_xlim(0, 1)
        ax1.xaxis.set_major_locator(mticker.FixedLocator([0.0, 0.4, 0.8, 1.0]))

        # (2) OOD hist
        ax2 = plt.subplot(2, 2, 2)
        ax2.hist(s0[y==1], bins=bins, alpha=0.35, density=True, label="OOD pre",   color="tab:blue")
        if s1 is not None: ax2.hist(s1[y==1], bins=bins, alpha=0.35, density=True, label="OOD inner", color="tab:orange")
        if s2 is not None: ax2.hist(s2[y==1], bins=bins, alpha=0.35, density=True, label="OOD outer", color="tab:green")
        ax2.set_title(f"{fam} — OOD hist"); ax2.set_xlim(0, 1)
        ax2.xaxis.set_major_locator(mticker.FixedLocator([0.0, 0.4, 0.8, 1.0]))

        # (3) ROC
        ax3 = plt.subplot(2, 1, 2)
        fpr0, tpr0, _ = roc_curve(y, s0); auc0 = auc(fpr0, tpr0) if len(fpr0) > 1 else float("nan")
        ax3.plot(fpr0, tpr0, color="tab:blue",   lw=2, label=f"pre (AUC={auc0:.3f})")
        if s1 is not None:
            fpr1, tpr1, _ = roc_curve(y, s1); auc1 = auc(fpr1, tpr1) if len(fpr1) > 1 else float("nan")
            ax3.plot(fpr1, tpr1, color="tab:orange", lw=2, label=f"inner (AUC={auc1:.3f})")
        if s2 is not None:
            fpr2, tpr2, _ = roc_curve(y, s2); auc2 = auc(fpr2, tpr2) if len(fpr2) > 1 else float("nan")
            ax3.plot(fpr2, tpr2, color="tab:green",  lw=2, label=f"outer (AUC={auc2:.3f})")
        ax3.plot([0, 1], [0, 1], "k--", lw=1)
        ax3.set_title(f"{fam} — ROC")
        ax3.set_xlim(0, 1); ax3.set_ylim(0, 1)
        ax3.xaxis.set_major_locator(mticker.FixedLocator([0.0, 0.4, 0.8, 1.0]))
        ax3.legend(loc="best")

        fig.tight_layout()
        out_path = os.path.join(save_dir, f"{fam}_combo.png")
        fig.savefig(out_path, dpi=200)   # no bbox_inches="tight" (keeps width small)
        plt.close(fig)

        saved.append(out_path)
        if verbose:
            print(f"[PLOT] Saved: {out_path}")

    print(f"[PLOT] Per-family overlays saved ({len(saved)} figs) → {save_dir}")

In [14]:


# ------------------------ Models ------------------------
class ResBlock(nn.Module):
    def __init__(self, d, drop_path=0.0):
        super().__init__()
        self.ln1 = nn.LayerNorm(d)
        self.fc1 = nn.Linear(d, d)
        self.ln2 = nn.LayerNorm(d)
        self.fc2 = nn.Linear(d, d)
        self.drop_path = float(drop_path)
    def forward(self, x):
        h = self.fc1(F.gelu(self.ln1(x)))
        h = self.fc2(F.gelu(self.ln2(h)))
        if self.training and self.drop_path>0 and torch.rand(()) < self.drop_path:
            return x
        return x + 0.5*h

class Encoder(nn.Module):
    def __init__(self, in_dim, hid=256, depth=3, p_drop=0.1, bottleneck=128, feat_drop=0.0):
        super().__init__()
        self.inp = nn.Linear(in_dim, hid)
        self.feat_drop = feat_drop
        self.blocks = nn.ModuleList([ResBlock(hid, drop_path=0.05) for _ in range(depth)])
        self.dropout = nn.Dropout(p_drop)
        self.out = nn.Linear(hid, bottleneck)
    def forward(self, x):
        if self.training and self.feat_drop>0:
            m = torch.rand_like(x) > self.feat_drop
            x = x * m
        h = F.gelu(self.inp(x))
        for blk in self.blocks: h = blk(h)
        h = self.dropout(h)
        return self.out(h)

class Decoder(nn.Module):
    def __init__(self, bottleneck=128, hid=256, out_dim=77, depth=2):
        super().__init__()
        self.fc = nn.Linear(bottleneck, hid)
        self.blocks = nn.ModuleList([ResBlock(hid) for _ in range(depth)])
        self.out = nn.Linear(hid, out_dim)
    def forward(self, z):
        h = F.gelu(self.fc(z))
        for blk in self.blocks: h = blk(h)
        return self.out(h)

class Classifier(nn.Module):
    def __init__(self, bottleneck=128, num_classes=2):
        super().__init__()
        self.ln = nn.LayerNorm(bottleneck)
        self.fc = nn.Linear(bottleneck, num_classes)
    def forward(self, z):
        return self.fc(F.gelu(self.ln(z)))

In [15]:
# ===== ODIN: epsilon-fix (replace the old pieces) ============================
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

class ODINParams(nn.Module):
    """
    Same API as before, but eps is parameterized to stay positive & bounded.
    """
    def __init__(self, T0, eps0, learn_T=True, learn_eps=False,
                 T_bounds=(0.5, 10.0), eps_bounds=(1e-5, 2e-2)):
        super().__init__()
        self.T_min, self.T_max = map(float, T_bounds)
        self.E_min, self.E_max = map(float, eps_bounds)

        def inv_sigmoid(x, a, b):
            x = float(np.clip((x - a) / (b - a), 1e-6, 1-1e-6))
            return np.log(x) - np.log(1 - x)

        # Temperature (bounded with sigmoid)
        self._T_raw = nn.Parameter(
            torch.tensor(inv_sigmoid(T0, self.T_min, self.T_max), dtype=torch.float32),
            requires_grad=bool(learn_T)
        )

        # Epsilon (softplus -> positive; softly clamped into [E_min, E_max])
        rho0 = np.log(np.exp(eps0 - self.E_min) - 1.0)  # inverse softplus(+E_min)
        self._eps_raw = nn.Parameter(
            torch.tensor(rho0, dtype=torch.float32),
            requires_grad=bool(learn_eps)
        )

    @property
    def T(self):
        return self.T_min + (self.T_max - self.T_min) * torch.sigmoid(self._T_raw)

    @property
    def eps(self):
        eps = F.softplus(self._eps_raw) + self.E_min
        return torch.clamp(eps, self.E_min, self.E_max)


def _odin_direction(x, encoder, clf, Tcur: torch.Tensor) -> torch.Tensor:
    """
    Build the ODIN/FGSM direction ∂/∂x of -log p_top. We only use its sign
    and we *detach the sign* so the direction is treated as a constant in the
    subsequent update (STE-style).
    """
    x_req = x.detach().clone().requires_grad_(True)
    logits = clf(encoder(x_req)) / Tcur
    p = logits.softmax(dim=1)
    top = p.max(dim=1).values
    loss = -torch.log(top.clamp_min(1e-12)).mean()
    g = torch.autograd.grad(loss, x_req, only_inputs=True)[0]
    return g.sign().detach()  # <-- direction is constant (no grads through sign)


def odin_perturb(x, encoder, clf, odin: ODINParams, steps: int = 1, tau: float = 0.0):
    """
    Multi-step ODIN where the *update* is differentiable w.r.t. eps/T.
    Only the direction is detached. No no_grad() around the update.
    If steps<=0, returns x unchanged.
    """
    if steps <= 0:
        return x

    Tcur = odin.T          # stays in-graph so T can learn if required
    eps  = odin.eps        # stays in-graph so eps can learn
    step = eps / max(1, steps)

    x0 = x
    x_adv = x
    for _ in range(steps):
        d = _odin_direction(x_adv, encoder, clf, Tcur)  # detached sign
        x_adv = x_adv + step * d                        # <-- differentiable w.r.t. eps/T
        if tau > 0.0:
            x_adv = (1.0 - tau) * x_adv + tau * x0     # optional EMA pullback

    return x_adv


def odin_forward_train(x: torch.Tensor, encoder, clf, odin: ODINParams, *, steps: int = 1):
    """
    Training forward:
      - builds x_adv with ODIN (update in-graph),
      - returns calibrated logits on x_adv.
    """
    if steps <= 0:
        z = encoder(x)
        return clf(z) / odin.T

    x_adv = odin_perturb(x, encoder, clf, odin, steps=steps)
    z = encoder(x_adv)
    return clf(z) / odin.T


@torch.no_grad()
def odin_score_eval(x: torch.Tensor, encoder, clf, odin: ODINParams, *, steps: int = 1) -> torch.Tensor:
    """
    Eval-time scorer: builds x_adv the same way but we don't need grads for eval.
    (It’s fine that eps/T don't get grads here.)
    Returns p(normal) = sigmoid(calibrated normal-class logit).
    """
    if steps <= 0:
        logits = clf(encoder(x)) / odin.T
        return torch.sigmoid(logits[:, 0])

    # Just reuse the same recipe under no_grad for speed
    Tcur = odin.T
    eps  = odin.eps
    step = eps / max(1, steps)

    x0 = x
    x_req = x.detach().clone().requires_grad_(True)
    logits0 = clf(encoder(x_req)) / Tcur
    top = logits0.softmax(dim=1).max(dim=1).values
    (-top.mean()).backward()
    d = x_req.grad.sign()  # no detach needed in eval
    x_adv = x + step * d
    logits = clf(encoder(x_adv)) / odin.T
    return torch.sigmoid(logits[:, 0])



def odin_normal_logit(x, encoder, clf, odin: ODINParams, *, steps=1):
    """
    EVAL-ONLY scorer: returns (calibrated) normal-class logit after 1-step ODIN.
    Side-effect free (no graph to params or eps).
    """
    assert steps >= 0
    if steps == 0:
        with torch.set_grad_enabled(False):
            z = encoder(x)
            logits = clf(z) / odin.T            # <-- calibrate
            return logits[:, 0]

    # 1-step FGSM direction (builds a tiny graph only to get input grads)
    x_adv = x.detach().clone().requires_grad_(True)
    z0 = encoder(x_adv)
    logits0 = clf(z0) / odin.T                  # <-- calibrate in the targeting step too
    logit_norm0 = logits0[:, 0]
    loss = -logit_norm0.sum()
    grads = torch.autograd.grad(loss, x_adv, only_inputs=True)[0]

    # update adversarial input WITHOUT building a graph to eps/params
    with torch.no_grad():
        x_adv = x_adv + odin.eps * _sign(grads)

    # final forward for the score (no grad)
    with torch.set_grad_enabled(False):
        z = encoder(x_adv)
        logits = clf(z) / odin.T                # <-- calibrate
        return logits[:, 0]


In [16]:

# pcgrad.py
import torch
from typing import Iterable, List, Tuple

@torch.no_grad()
def _flatten_grads(param_list: Iterable[torch.nn.Parameter]) -> List[torch.Tensor]:
    grads = []
    for p in param_list:
        if p.grad is None:
            grads.append(torch.zeros_like(p))
        else:
            grads.append(p.grad.clone())
    return grads

def _set_grads(param_list: Iterable[torch.nn.Parameter], grads: List[torch.Tensor]):
    for p, g in zip(param_list, grads):
        p.grad = g

def _zero_grads(param_list: Iterable[torch.nn.Parameter]):
    for p in param_list:
        if p.grad is not None:
            p.grad.zero_()

def _add_inplace(x, y, alpha=1.0):
    for xi, yi in zip(x, y):
        xi.add_(yi, alpha=alpha)

def _dot(x: List[torch.Tensor], y: List[torch.Tensor]) -> torch.Tensor:
    return torch.stack([torch.dot(a.flatten(), b.flatten()) for a, b in zip(x, y)]).sum()

def _norm2(x: List[torch.Tensor]) -> torch.Tensor:
    return torch.sqrt(_dot(x, x) + 1e-12)

from typing import Sequence

def pcgrad_summed(params, losses):
    """
    Compute per-loss grads, project away conflicts (PCGrad), and write the
    summed projected grads into params[i].grad. No optimizer steps here.

    params: list of nn.Parameter that require_grad
    losses: list of scalar tensors
    """
    # collect grads per loss
    per_loss_grads = []
    for L in losses:
        grads = torch.autograd.grad(L, params, retain_graph=True, allow_unused=True)
        per_loss_grads.append([
            (g.detach() if g is not None else torch.zeros_like(p, requires_grad=False))
            for g, p in zip(grads, params)
        ])

    # project each task grad against others when dot < 0
    proj_grads = []
    for i, gi in enumerate(per_loss_grads):
        gi_flat = [g.view(-1) for g in gi]
        gi_cat  = torch.cat(gi_flat)
        gi_mod  = gi_cat.clone()
        for j, gj in enumerate(per_loss_grads):
            if j == i:
                continue
            gj_flat = [g.view(-1) for g in gj]
            gj_cat  = torch.cat(gj_flat)
            dot = torch.dot(gi_mod, gj_cat)
            if dot < 0:
                # project gi_mod onto the normal plane of gj
                denom = torch.dot(gj_cat, gj_cat) + 1e-12
                gi_mod = gi_mod - (dot / denom) * gj_cat
        # reshape back to param shapes
        off = 0
        gi_proj = []
        for p, g in zip(params, gi):
            n = g.numel()
            gi_proj.append(gi_mod[off:off+n].view_as(p))
            off += n
        proj_grads.append(gi_proj)

    # sum projected grads and assign to .grad
    for p in params:
        p.grad = None
    for gi_proj in proj_grads:
        for p, g in zip(params, gi_proj):
            if p.grad is None:
                p.grad = g.clone()
            else:
                p.grad.add_(g)



# metrics.py
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, average_precision_score


def compute_micro_macro(y_true, y_score, y_pred, fam_ids):
    """
    y_true: (N,) 0/1
    y_score: (N,) anomaly score or probability for ROC/AP
    y_pred: (N,) 0/1 at your chosen threshold
    fam_ids: (N,) int ids per sample
    """
    # ---- MICRO (pooled)
    prec_mi, rec_mi, f1_mi, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
    auc_mi = roc_auc_score(y_true, y_score)
    ap_mi  = average_precision_score(y_true, y_score)

    # ---- MACRO (avg of per-family)
    fams = np.unique(fam_ids)
    precs, recs, f1s, aucs, aps = [], [], [], [], []
    for f in fams:
        m = (fam_ids == f)
        if m.sum() < 2 or len(np.unique(y_true[m])) < 2:
            continue
        p_f, r_f, f_f, _ = precision_recall_fscore_support(y_true[m], y_pred[m], average='binary', zero_division=0)
        precs.append(p_f); recs.append(r_f); f1s.append(f_f)
        aucs.append(roc_auc_score(y_true[m], y_score[m]))
        aps.append(average_precision_score(y_true[m], y_score[m]))
    macro = dict(
        precision=np.mean(precs) if precs else 0.0,
        recall=np.mean(recs) if recs else 0.0,
        f1=np.mean(f1s) if f1s else 0.0,
        auc=np.mean(aucs) if aucs else 0.0,
        ap=np.mean(aps) if aps else 0.0,
    )
    micro = dict(precision=prec_mi, recall=rec_mi, f1=f1_mi, auc=auc_mi, ap=ap_mi)
    return micro, macro

## ======================== Utilities used by the trainers ========================
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# assumes these exist in your file:
# - device
# - NUM_WORKERS, PIN_MEMORY
# - INNER_LR_MAX, WEIGHT_DECAY, CLIP_NORM, USE_AMP, LAMBDA_REC
# - OUTER_MARGIN_M, OUTER_MARGIN_ALPHA
# - OUTER_BATCH, OUTER_EPOCHS, TRAIN_PGD_STEPS, ODIN_TAU
# - pcgrad_summed(params, losses)  # your PCGrad helper
# - odin_normal_logit(x, encoder, clf, odin, steps, tau) -> [B,1] logits on "normal" class

def _safe_grad_norm(p: torch.nn.Parameter) -> float:
    g = getattr(p, "grad", None)
    if g is None:
        return 0.0
    if not torch.isfinite(g).all():
        return 0.0
    return float(g.detach().norm().item())

In [17]:
# ------------------------ Inner Trainer (with benign-center EMA) ------------------------
# ======================== Utilities used by the trainers ========================
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# expects in your file:
# - device
# - NUM_WORKERS, PIN_MEMORY
# - INNER_LR_MAX, WEIGHT_DECAY, CLIP_NORM, USE_AMP, LAMBDA_REC
# - OUTER_MARGIN_M, OUTER_MARGIN_ALPHA
# - OUTER_BATCH, OUTER_EPOCHS, TRAIN_PGD_STEPS, ODIN_TAU
# - pcgrad_summed(params, losses)         # your PCGrad helper
# - odin_normal_logit(x, encoder, clf, odin, steps, tau) -> [B,1] normal-class logits

def _safe_grad_norm(p: torch.nn.Parameter) -> float:
    g = getattr(p, "grad", None)
    if g is None or not torch.isfinite(g).all():
        return 0.0
    return float(g.detach().norm().item())

In [18]:


# ------------------------ Inner Trainer (with benign-center EMA) ------------------------
# ------------------------ Outer Trainer (hardened against NaNs) ------------------------
# ------------------------ Inner Trainer (unchanged) ------------------------
class InnerTrainer:
    def __init__(self, encoder: nn.Module, decoder: nn.Module, clf: nn.Module,
                 lr_max: float = INNER_LR_MAX, weight_decay: float = WEIGHT_DECAY,
                 clip_norm: float = CLIP_NORM, use_amp: bool = USE_AMP,
                 lambda_rec: float = LAMBDA_REC, total_inner_steps_est: int | None = None):
        self.encoder, self.decoder, self.clf = encoder, decoder, clf
        self.clip_norm = clip_norm
        self.use_amp = use_amp and (device.type == "cuda")
        self.lambda_rec = lambda_rec
        params = list(self.encoder.parameters()) + list(self.decoder.parameters()) + list(self.clf.parameters())
        for p in params: p.requires_grad_(True)
        self.opt = torch.optim.AdamW(params, lr=lr_max, weight_decay=weight_decay, betas=(0.9, 0.99))
        if total_inner_steps_est is None:
            total_inner_steps_est = 50_000
        self.sched = torch.optim.lr_scheduler.CosineAnnealingLR(self.opt, T_max=total_inner_steps_est)
        self.scaler = torch.amp.GradScaler('cuda', enabled=self.use_amp)

    def train_one_episode(self, meta_tasks: list, batch_size: int = INNER_BATCH, epochs: int = INNER_EPOCHS):
        # enable grads
        for p in self.encoder.parameters(): p.requires_grad_(True)
        for p in self.decoder.parameters(): p.requires_grad_(True)
        for p in self.clf.parameters():     p.requires_grad_(True)
        self.encoder.train(); self.decoder.train(); self.clf.train()
        device_ = next(self.encoder.parameters()).device

        # build loaders per task
        task_loaders = []
        for (Xi, yi, _) in meta_tasks:
            Xi_t = torch.from_numpy(Xi.astype(np.float32, copy=False))
            yi_t = torch.from_numpy(yi.astype(np.int64,  copy=False))
            ds = TensorDataset(Xi_t, yi_t)
            dl = DataLoader(ds, batch_size=batch_size, shuffle=True, drop_last=False,
                            num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
            task_loaders.append(dl)
        if len(task_loaders) == 0:
            print("[INNER/BCE] No data in meta_tasks; skipping.")
            return

        steps_per_epoch = max(1, max(len(dl) for dl in task_loaders))
        def _fresh_iters(): return [iter(dl) for dl in task_loaders]
        iters = _fresh_iters()
        params = list(self.encoder.parameters()) + list(self.decoder.parameters()) + list(self.clf.parameters())

        for ep in range(1, epochs + 1):
            epoch_losses = []
            for _ in range(steps_per_epoch):
                batches = []
                for ti, it in enumerate(iters):
                    try:
                        xb, yb = next(it)
                    except StopIteration:
                        iters[ti] = iter(task_loaders[ti])
                        try:
                            xb, yb = next(iters[ti])
                        except StopIteration:
                            continue
                    batches.append((xb.to(device_).float(), yb.to(device_).long()))
                if len(batches) == 0:
                    continue

                xb = torch.cat([b[0] for b in batches], dim=0)
                yb = torch.cat([b[1] for b in batches], dim=0)
                tgt = (yb == 0).float()  # normal=1, anomaly=0 → target for normal-logit

                self.opt.zero_grad(set_to_none=True)
                with torch.amp.autocast('cuda', enabled=self.use_amp):
                    z = self.encoder(xb)
                    logits = self.clf(z)            # [B, 2]
                    logit_norm = logits[:, 0]       # normal-class logit
                    bce = F.binary_cross_entropy_with_logits(logit_norm, tgt)
                    loss = bce
                    if self.lambda_rec > 0:
                        xhat = self.decoder(z)
                        loss = loss + self.lambda_rec * F.mse_loss(xhat, xb)

                self.scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(params, max_norm=self.clip_norm)
                self.scaler.step(self.opt)
                self.scaler.update()
                self.sched.step()

                epoch_losses.append(float(loss.detach().item()))
            mean_loss = np.mean(epoch_losses) if epoch_losses else 0.0
            print(f"[INNER/BCE] Ep{ep:02d} | Loss={mean_loss:.5f} | lr={self.opt.param_groups[0]['lr']:.2e}")



In [19]:



# ------------------------ Outer Trainer (PCGrad + simple margin, clean logging) ------------------------
def _safe_grad_norm(p):
    if p is None or (not hasattr(p, "grad")) or p.grad is None:
        return 0.0
    g = p.grad.detach()
    return float(g.norm().item())

# ------------------------ Outer Trainer (PCGrad + simple margin; ODIN gradients fixed) ------------------------
from collections import defaultdict

# ======================== OuterTrainer (ε learnable, margin, PCGrad) ========================
# ------------------------ Outer Trainer (PCGrad + learnable T and optional ε) ------------------------
# ------------------------ Outer Trainer (ODIN + PCGrad + margin; T and eps learnable) ------------------------
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from typing import List

# ------------------------ Outer Trainer (recompute for ODIN; no inplace graph reuse) ------------------------
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from typing import List

import numpy as np
from torch.utils.data import TensorDataset, DataLoader

 #---------- OuterTrainer ----------
class OuterTrainer:
    def __init__(
        self,
        encoder: nn.Module,
        clf: nn.Module,
        odin,                              # ODINParams-like object
        *,
        lr_odin: float = 1e-3,
        lr_phi: float  = 3e-3,
        wd_odin: float = 0.0,
        wd_phi: float  = 1e-4,
        total_outer_steps_est: int = 10_000,
        unfreeze_encoder: bool = True,
        enc_lr: float = 1e-4,
        enc_wd: float = 1e-4,
        logT_l2: float = 0.0,              # prior on raw T param (if present)
        logeps_l2: float = 0.0,            # prior on raw eps param (if present)
        amp: bool = True,
        margin_m: float = 0.50,
        margin_alpha: float = 0.10
    ):
        super().__init__()
        self.encoder, self.clf, self.odin = encoder, clf, odin
        self.use_amp = amp and torch.cuda.is_available()
        self.margin_m = float(margin_m)
        self.margin_alpha = float(margin_alpha)
        self.logT_l2 = float(logT_l2)
        self.logeps_l2 = float(logeps_l2)

        # -------- shared params (head + optional last encoder block) --------
        shared_params = [p for p in clf.parameters() if p.requires_grad]
        self._enc_last = []
        if unfreeze_encoder and hasattr(self.encoder, "blocks") and isinstance(self.encoder.blocks, nn.ModuleList) and len(self.encoder.blocks) > 0:
            for p in self.encoder.blocks[-1].parameters():
                p.requires_grad_(True)
                self._enc_last.append(p)
        shared_params += self._enc_last
        if not shared_params:
            raise RuntimeError("OuterTrainer: No shared (head/last-encoder-block) params to optimize.")
        self.shared_params = shared_params

        self.opt_shared = torch.optim.Adam([
            {"params": [p for p in self.clf.parameters() if p.requires_grad], "lr": lr_phi, "weight_decay": wd_phi},
            *([{"params": self._enc_last, "lr": enc_lr, "weight_decay": enc_wd}] if self._enc_last else [])
        ])
        self.sched_shared = torch.optim.lr_scheduler.CosineAnnealingLR(self.opt_shared, T_max=total_outer_steps_est)

        # -------- ODIN params (support either new _T_raw/_eps_raw or old logT/logeps) --------
        odin_params = []
        self._T_raw   = getattr(self.odin, "_T_raw",   None)
        self._eps_raw = getattr(self.odin, "_eps_raw", None)
        self._logT    = getattr(self.odin, "logT",     None)    # legacy (if any)
        self._logeps  = getattr(self.odin, "logeps",   None)    # legacy (if any)

        for cand in (self._T_raw, self._eps_raw, self._logT, self._logeps):
            if isinstance(cand, torch.nn.Parameter) and cand.requires_grad:
                odin_params.append(cand)

        self.opt_odin = torch.optim.Adam([{"params": odin_params, "lr": lr_odin, "weight_decay": wd_odin}]) if odin_params else None
        self.sched_odin = torch.optim.lr_scheduler.CosineAnnealingLR(self.opt_odin, T_max=total_outer_steps_est) if self.opt_odin else None

    def _freeze_encoder_except_last(self):
        self.encoder.eval()
        for p in self.encoder.parameters(): p.requires_grad_(False)
        for p in self._enc_last: p.requires_grad_(True)

    def _raw_T_param(self):
        # whichever exists/learns; used for priors & grad-norm printing
        return self._T_raw if isinstance(self._T_raw, torch.nn.Parameter) else self._logT

    def _raw_eps_param(self):
        return self._eps_raw if isinstance(self._eps_raw, torch.nn.Parameter) else self._logeps


    def train_one_episode(
            self,
            Xq: np.ndarray,
            yq: np.ndarray,
            *,
            batch_size: int,
            epochs: int,
            steps: int,
            shuffle: bool = True,
            num_workers: int = 0,
            pin_memory: bool = False,
        ):
        """
        Outer loop episode:
          - Builds class-balanced mini-batches (ID/OOD).
          - L_inner: clean-ID push-up (no ODIN perturb).
          - L_outer: ODIN (on both ID & OOD) with BCE on normal logit + margin hinge.
          - PCGrad over [L_inner, L_outer] for the shared params (encoder+clf).
          - Optional second pass to update ODIN hyper-params (T, eps) only.
        """
        dev = next(self.clf.parameters()).device

        # freeze all encoder layers except the last block, if you expose that helper
        self._freeze_encoder_except_last()
        self.clf.train()
        if hasattr(self.odin, "train"):
            self.odin.train()

        # DataLoader (torch tensors)
        ds = TensorDataset(
            torch.from_numpy(Xq.astype(np.float32, copy=False)),
            torch.from_numpy(yq.astype(np.int64,  copy=False)),
        )
        dl = DataLoader(
            ds, batch_size=batch_size, shuffle=shuffle, drop_last=False,
            num_workers=num_workers, pin_memory=pin_memory
        )

        for ep in range(1, epochs + 1):
            tot_loss, pairs = 0.0, 0

            for xb, yb in dl:
                xb = xb.to(dev, non_blocking=True).float()
                yb = yb.to(dev, non_blocking=True).long()

                # balance ID/OOD within the batch
                idm, oodm = (yb == 0), (yb == 1)
                if idm.sum() == 0 or oodm.sum() == 0:
                    continue
                k = int(min(int(idm.sum()), int(oodm.sum())))
                xb_id  = xb[idm][:k]
                xb_ood = xb[oodm][:k]

                # ---------------- PASS A: Shared params (PCGrad over [L_inner, L_outer]) ----------------
                with torch.amp.autocast("cuda", enabled=self.use_amp):
                    # ----- OUTER: ODIN logits (train-time ODIN forward) -----
                    # IMPORTANT: the new odin_forward_train builds x_adv with the update
                    # in-graph, so grads flow to eps/T. Only the FGSM sign is detached.
                    logits_id_odin  = odin_forward_train(xb_id,  self.encoder, self.clf, self.odin, steps=steps)
                    logits_ood_odin = odin_forward_train(xb_ood, self.encoder, self.clf, self.odin, steps=steps)

                    # Use the 'normal' logit (index 0) as anomaly score proxy (higher => more normal)
                    logit_id_norm  = logits_id_odin[:, 0]
                    logit_ood_norm = logits_ood_odin[:, 0]

                    # BCE target: ID->1, OOD->0 for the normal logit
                    logits_cat = torch.cat([logit_id_norm, logit_ood_norm], dim=0)  # [2k]
                    targets    = torch.cat([
                        torch.ones_like(logit_id_norm),
                        torch.zeros_like(logit_ood_norm)
                    ], dim=0).float()
                    bce = F.binary_cross_entropy_with_logits(logits_cat, targets)

                    # Margin on normal-logit gap (ID minus OOD)
                    gap = (logit_id_norm.mean() - logit_ood_norm.mean())
                    margin_loss = F.relu(self.margin_m - gap)
                    L_outer = bce + self.margin_alpha * margin_loss

                    # Optional priors (on *raw* parameters) to keep T/eps in check
                    rawT = self._raw_T_param()
                    rawe = self._raw_eps_param()
                    if self.logT_l2 > 0.0 and isinstance(rawT, torch.nn.Parameter) and rawT.requires_grad:
                        L_outer = L_outer + self.logT_l2 * (rawT ** 2)
                    if self.logeps_l2 > 0.0 and isinstance(rawe, torch.nn.Parameter) and rawe.requires_grad:
                        L_outer = L_outer + self.logeps_l2 * (rawe ** 2)

                    # ----- INNER: clean-ID push-up (no ODIN) -----
                    z_id_clean = self.encoder(xb_id)
                    logits_id_clean = self.clf(z_id_clean)  # [k,2]
                    logit_id_clean_norm = logits_id_clean[:, 0]
                    L_inner = F.binary_cross_entropy_with_logits(
                        logit_id_clean_norm,
                        torch.ones_like(logit_id_clean_norm)
                    )

                # PCGrad on shared params
                self.opt_shared.zero_grad(set_to_none=True)
                pcgrad_summed(self.shared_params, [L_inner, L_outer])
                torch.nn.utils.clip_grad_norm_(self.shared_params, max_norm=1.0)
                self.opt_shared.step()
                self.sched_shared.step()

                # ---------------- PASS B: ODIN params only (optional) ----------------
                if self.opt_odin is not None:
                    # temporarily freeze shared params so only ODIN gets grads
                    for p in self.shared_params:
                        p.requires_grad_(False)
                    try:
                        self.opt_odin.zero_grad(set_to_none=True)
                        with torch.amp.autocast("cuda", enabled=self.use_amp):
                            li  = odin_forward_train(xb_id,  self.encoder, self.clf, self.odin, steps=steps)
                            lo  = odin_forward_train(xb_ood, self.encoder, self.clf, self.odin, steps=steps)
                            li_n = li[:, 0]; lo_n = lo[:, 0]

                            lgs = torch.cat([li_n, lo_n], dim=0)
                            tgs = torch.cat([torch.ones_like(li_n), torch.zeros_like(lo_n)], dim=0).float()
                            bce2 = F.binary_cross_entropy_with_logits(lgs, tgs)

                            gap2 = (li_n.mean() - lo_n.mean())
                            margin2 = F.relu(self.margin_m - gap2)

                            L_odin = bce2 + self.margin_alpha * margin2
                            rawT = self._raw_T_param()
                            rawe = self._raw_eps_param()
                            if self.logT_l2 > 0.0 and isinstance(rawT, torch.nn.Parameter) and rawT.requires_grad:
                                L_odin = L_odin + self.logT_l2 * (rawT ** 2)
                            if self.logeps_l2 > 0.0 and isinstance(rawe, torch.nn.Parameter) and rawe.requires_grad:
                                L_odin = L_odin + self.logeps_l2 * (rawe ** 2)

                        L_odin.backward()
                        # clip just ODIN params
                        to_clip = []
                        for g in self.opt_odin.param_groups:
                            to_clip += g["params"]
                        torch.nn.utils.clip_grad_norm_(to_clip, max_norm=1.0)

                        self.opt_odin.step()
                        if self.sched_odin is not None:
                            self.sched_odin.step()
                    finally:
                        # unfreeze shared params
                        for p in self.shared_params:
                            p.requires_grad_(True)

                # bookkeeping for logging (use the BCE as representative loss)
                tot_loss += float(bce.detach().item()) * k
                pairs += k

            # ---- episode logging ----
            avg = tot_loss / max(1, pairs)

            # nice printing of T, eps and their grad norms (if learnable)
            def _to_float(x):
                if isinstance(x, torch.Tensor): return float(x.detach().cpu())
                return float(x)

            T_show = _to_float(self.odin.T)
            E_show = _to_float(self.odin.eps)

            gT = 0.0
            rawT = self._raw_T_param()
            if isinstance(rawT, torch.nn.Parameter) and rawT.grad is not None:
                gT = float(rawT.grad.detach().norm().item())

            gE = 0.0
            rawe = self._raw_eps_param()
            if isinstance(rawe, torch.nn.Parameter) and rawe.grad is not None:
                gE = float(rawe.grad.detach().norm().item())

            lr_shared = self.opt_shared.param_groups[0]["lr"]
            lr_odin   = (self.opt_odin.param_groups[0]["lr"] if self.opt_odin is not None else 0.0)

            print(
                f"[OUTER/ODIN-BCE+PCGrad] Ep{ep:02d} | Loss={avg:.5f} "
                f"T={T_show:.6f} eps={E_show:.6e} "
                f"margin_m={self.margin_m:.3f} alpha={self.margin_alpha:.3f} "
                f"||∇rawT||={gT:.3e} ||∇raweps||={gE:.3e} "
                f"lr_shared={lr_shared:.2e} lr_odin={lr_odin:.2e}"
            )

        # unfreeze everything for the next (inner) stage
        for p in self.encoder.parameters(): p.requires_grad_(True)
        for p in self.clf.parameters():     p.requires_grad_(True)


In [20]:



#------- Episode builders ------------------------
def _draw_k_up_to(rng: np.random.Generator, k_avail: int, k_max: int, k_min: int) -> int:
    if k_avail <= 0: return 0
    hi = min(k_max, k_avail)
    if hi < k_min: return 0
    return int(rng.integers(k_min, hi + 1))

def select_inner_fams_up_to(cache, rng, allowed_fams, part, inner_max_fams, inner_min_fams=2):
    cands = [f for f in cache.families_with(cls=1, part=part) if f in allowed_fams]
    rng.shuffle(cands)
    k = _draw_k_up_to(rng, len(cands), inner_max_fams, inner_min_fams)
    return cands[:k]

def select_outer_ood_fams_up_to(cache, rng, inner_used, allowed_fams, part, outer_max_fams, outer_min_fams=1):
    # don't reuse inner families as OOD; rotate randomly for diversity
    cands = [f for f in cache.families_with(cls=1, part=part) if (f in allowed_fams) and (f not in inner_used)]
    rng.shuffle(cands)
    k = _draw_k_up_to(rng, len(cands), outer_max_fams, outer_min_fams)
    return cands[:k]

def _balanced_downsample(X: np.ndarray, y: np.ndarray, target_total: Optional[int], rng: np.random.Generator):
    if (target_total is None) or (target_total <= 0) or (X.shape[0] <= target_total):
        return X, y
    half = target_total // 2
    idx_pos = np.where(y == 1)[0]; idx_neg = np.where(y == 0)[0]
    take_pos = min(len(idx_pos), half); take_neg = min(len(idx_neg), half)
    short = 2*half - (take_pos + take_neg)
    if short > 0:
        if take_pos < half: take_neg = min(len(idx_neg), take_neg + short)
        elif take_neg < half: take_pos = min(len(idx_pos), take_pos + short)
    sel_pos = idx_pos if take_pos >= len(idx_pos) else rng.choice(idx_pos, size=take_pos, replace=False)
    sel_neg = idx_neg if take_neg >= len(idx_neg) else rng.choice(idx_neg, size=take_neg, replace=False)
    sel = np.concatenate([sel_pos, sel_neg]); rng.shuffle(sel)
    X2 = X[sel].astype(np.float32, copy=False); y2 = y[sel].astype(np.int64, copy=False)
    return X2, y2

def build_inner_episode_batch(cache, rng, part: str, fams: List[str], pos_per_fam: int, neg_per_fam):
    X_parts, y_parts, used = [], [], []
    for fam in fams:
        Xa = cache.sample(fam, 1, part, pos_per_fam, rng)
        if Xa.size == 0: continue
        kneg = Xa.shape[0] if (isinstance(neg_per_fam, str) and neg_per_fam == "match") else int(neg_per_fam)
        kneg = max(1, kneg)
        Xn = cache.sample(fam, 0, part, kneg, rng)
        if Xn.size == 0: continue
        X_parts += [Xa, Xn]
        y_parts += [np.ones(Xa.shape[0], np.int64), np.zeros(Xn.shape[0], np.int64)]
        used.append(fam)
    if not X_parts:
        return np.empty((0,0), np.float32), np.empty((0,), np.int64), []
    X = np.vstack(X_parts).astype(np.float32, copy=False)
    y = np.concatenate(y_parts, axis=0)
    X, y = _balanced_downsample(X, y, target_total=INNER_EPISODE_MAX, rng=rng)
    return X, y, used


def build_outer_episode_batch(
    cache,
    rng: np.random.Generator,
    *,
    ood_fams: list,
    part: str = "train",
    # few-shot knobs
    fs_num_ood_classes: int = 2,
    fs_k_per_class: int = 32,
    allow_smaller: bool = True,
):
    """
    Few-shot OUTER batch builder (cache-agnostic, uses DataCache.sample):
      • Choose up to `fs_num_ood_classes` families from `ood_fams`.
      • For each chosen fam, sample K anomalies (label=1) AND K normals (label=0) from the SAME family/split.
      • Concatenate, balance, and shuffle to return (Xq, yq, used_families).

    Returns:
        Xq: (N, D) float32
        yq: (N,) int64  with 1=OOD(anomaly), 0=ID(normal)
        used: list[str] of families actually used
    """
    if not ood_fams:
        D = cache.X[next(iter(cache.X))].shape[1]
        return np.empty((0, D), np.float32), np.empty((0,), np.int64), []

    # Pick families for few-shot (cap at fs_num_ood_classes)
    pool = list(ood_fams)
    rng.shuffle(pool)
    chosen = pool[: min(fs_num_ood_classes, len(pool))]

    X_pos_chunks, X_neg_chunks, used = [], [], []
    for fam in chosen:
        # How many are available on this split?
        avail_pos = cache.count(fam, 1, part)  # anomalies
        avail_neg = cache.count(fam, 0, part)  # normals
        if avail_pos <= 0 or avail_neg <= 0:
            continue

        k_a = min(fs_k_per_class, avail_pos)
        k_n = min(fs_k_per_class, avail_neg)

        if not allow_smaller and (k_a < fs_k_per_class or k_n < fs_k_per_class):
            # require full-K; skip if not enough
            continue
        if k_a == 0 or k_n == 0:
            continue

        Xa = cache.sample(fam, 1, part, k_a, rng)  # anomalies (+1)
        Xn = cache.sample(fam, 0, part, k_n, rng)  # normals   ( 0)

        if Xa.size == 0 or Xn.size == 0:
            continue

        X_pos_chunks.append(Xa.astype(np.float32, copy=False))
        X_neg_chunks.append(Xn.astype(np.float32, copy=False))
        used.append(fam)

    # Nothing usable → empty outer batch
    if len(X_pos_chunks) == 0 or len(X_neg_chunks) == 0:
        D = cache.X[next(iter(cache.X))].shape[1]
        return np.empty((0, D), np.float32), np.empty((0,), np.int64), []

    # Concatenate and class-balance (equal #pos/#neg)
    X_pos = np.concatenate(X_pos_chunks, axis=0).astype(np.float32, copy=False)
    X_neg = np.concatenate(X_neg_chunks, axis=0).astype(np.float32, copy=False)
    m = min(len(X_pos), len(X_neg))
    ip = rng.choice(len(X_pos), size=m, replace=False)
    in_ = rng.choice(len(X_neg), size=m, replace=False)

    Xq = np.concatenate([X_pos[ip], X_neg[in_]], axis=0)
    yq = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)], axis=0)

    # Shuffle the batch
    perm = rng.permutation(len(Xq))
    Xq, yq = Xq[perm], yq[perm]

    return Xq, yq, used

In [21]:


# ------------------------ Metrics / eval & Calibration ------------------------
@torch.no_grad()
def odin_pnorm_numpy(X_np, encoder, clf, odin, bs: int = 8192, steps: int = VAL_PGD_STEPS):
    encoder.eval(); clf.eval(); odin.eval()
    X_np = X_np.astype(np.float32, copy=False)
    outs = []
    for i in range(0, X_np.shape[0], bs):
        xb = torch.from_numpy(X_np[i:i+bs]).to(device).float()
        with torch.enable_grad():
            x_adv = odin_perturb(xb, encoder, clf, odin, steps=steps, tau=ODIN_TAU)
        logits = clf(encoder(x_adv)) / odin.T
        p_norm = torch.sigmoid(logits[:, 0])
        outs.append(p_norm.detach().cpu().numpy())
    return np.concatenate(outs, axis=0)


def _best_f1_with_min_precision(y_true: np.ndarray, scores_anom: np.ndarray, min_prec: float) -> Tuple[float, float]:
    # Scan a dense grid like _best_f1_threshold but require precision >= min_prec
    grid = np.unique(np.concatenate([np.linspace(1e-6, 0.05, 80),
                                     np.linspace(0.05, 0.95, 181),
                                     np.linspace(0.95, 0.999999, 50)]))
    best_t, best_f1 = None, -1.0
    for t in grid:
        yhat = (scores_anom >= t).astype(int)
        prec = precision_score(y_true, yhat, zero_division=0)
        if prec + 1e-12 < float(min_prec):  # fails constraint
            continue
        f1 = f1_score(y_true, yhat, zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, float(t)
    # Fallback: if constraint yields nothing, fall back to unconstrained best F1
    if best_t is None:
        return _best_f1_threshold(y_true, scores_anom)
    return best_t, best_f1


def _best_f1_threshold(y_true: np.ndarray, scores_anom: np.ndarray) -> Tuple[float,float]:
    grid = np.unique(np.concatenate([np.linspace(1e-6, 0.05, 80),
                                     np.linspace(0.05, 0.95, 181),
                                     np.linspace(0.95, 0.999999, 50)]))
    best_t, best_f1 = 0.5, -1.0
    for t in grid:
        yhat = (scores_anom >= t).astype(int)
        f1 = f1_score(y_true, yhat, zero_division=0)
        if f1 > best_f1: best_f1, best_t = f1, float(t)
    return best_t, best_f1

def _balanced_block(cache: DataCache, fam: str, rng: np.random.Generator, *, part: str, cap: int):
    m = min(cache.count(fam, 1, part), cache.count(fam, 0, part), cap)
    if m <= 0: return np.empty((0,0), np.float32), np.empty((0,), np.int64)
    Xa = cache.sample(fam, 1, part, m, rng); Xn = cache.sample(fam, 0, part, m, rng)
    X  = np.vstack([Xa, Xn]).astype(np.float32, copy=False)
    y  = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)])
    return X, y

# ------------------------ VAL metrics after each episode ------------------------
def _metrics_at_threshold(y_true: np.ndarray, scores_anom: np.ndarray, thr: float):
    """scores_anom: higher = more anomalous"""
    yhat = (scores_anom >= thr).astype(int)
    prec = precision_score(y_true, yhat, zero_division=0)
    rec  = recall_score(y_true, yhat, zero_division=0)
    acc  = accuracy_score(y_true, yhat)
    f1   = f1_score(y_true, yhat, zero_division=0)
    try:
        auc_roc = roc_auc_score(y_true, scores_anom) if (np.unique(y_true).size == 2) else float("nan")
    except Exception:
        auc_roc = float("nan")
    ap = average_precision_score(y_true, scores_anom) if y_true.sum() > 0 else float("nan")
    # PR-AUC (area under PR curve). AP is already the average precision; we show both.
    try:
        P, R, _ = precision_recall_curve(y_true, scores_anom)
        pr_auc = auc(R, P)
    except Exception:
        pr_auc = float("nan")
    return dict(prec=prec, rec=rec, acc=acc, f1=f1, auc_roc=auc_roc, ap=ap, pr_auc=pr_auc)

@torch.no_grad()
def log_val_stats_after_episode(
    cache: DataCache,
    encoder: nn.Module,
    clf: nn.Module,
    odin: ODINParams,
    rng: np.random.Generator,
    *,
    part: str = "val",
    cap_per_class: int = VAL_PER_CLASS_CAP,
    steps: int = VAL_PGD_STEPS,
    tag: str = ""
):
    """
    Computes pooled and per-family metrics on `part` (val/test) using ODIN scores.
    - Picks a single global threshold by maximizing F1 on the *pooled* set.
    - Reports pooled metrics and per-family metrics at that global threshold.
    - Uses up to `cap_per_class` ID and OOD per family (balanced block).
    """
    fams = sorted(cache.files.keys())

    # Build pooled evaluation set
    X_blocks, y_blocks, used_counts = [], [], {}
    for fam in fams:
        m = min(cache.count(fam, 1, part), cache.count(fam, 0, part), cap_per_class)
        if m <= 0:
            continue
        Xa = cache.sample(fam, 1, part, m, rng)
        Xn = cache.sample(fam, 0, part, m, rng)
        Xf = np.vstack([Xa, Xn]).astype(np.float32, copy=False)
        yf = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)])
        X_blocks.append(Xf); y_blocks.append(yf)
        used_counts[fam] = (m, m)

    if not X_blocks:
        print("[VAL-METRICS] no data for split:", part)
        return None

    X_all = np.vstack(X_blocks)
    y_all = np.concatenate(y_blocks)

    # Get ODIN "normal prob" then convert to anomaly score
    p_norm = odin_pnorm_numpy(X_all, encoder, clf, odin, steps=steps)
    scores = 1.0 - p_norm  # higher = more anomalous

    # Pick pooled best-F1 threshold and compute pooled metrics
    thr_star, _ = _best_f1_threshold(y_all, scores)
    pooled = _metrics_at_threshold(y_all, scores, thr_star)

    print(f"[VAL-METRICS] {part.upper()} {tag} | thr*={thr_star:.6f} | "
          f"Prec={pooled['prec']:.3f} Rec={pooled['rec']:.3f} Acc={pooled['acc']:.3f} "
          f"F1={pooled['f1']:.3f} AP={pooled['ap']:.3f} PR-AUC={pooled['pr_auc']:.3f} "
          f"AUC-ROC={pooled['auc_roc']:.3f}")

    # Per-family metrics @ the same global threshold
    print("[VAL-METRICS per-family @ global thr*]")
    off = 0
    for fam in fams:
        if fam not in used_counts:
            print(f"  📉 {fam:<12} | (no data)")
            continue
        m_id, m_ood = used_counts[fam]
        n = m_id + m_ood
        yf = y_all[off:off+n]
        sf = scores[off:off+n]
        off += n
        stats = _metrics_at_threshold(yf, sf, thr_star)
        print(f"  📊 {fam:<12} | Prec={stats['prec']:.3f} Rec={stats['rec']:.3f} "
              f"Acc={stats['acc']:.3f} F1={stats['f1']:.3f} "
              f"AP={stats['ap']:.3f} PR-AUC={stats['pr_auc']:.3f} "
              f"AUC-ROC={stats['auc_roc']:.3f}  (used ID={m_id:,} OOD={m_ood:,})")

    # Return in case you want to reuse the threshold elsewhere in your loop
    return float(thr_star), pooled



def test_per_family(cache: DataCache, encoder, clf, odin, thr_star: float, rng: np.random.Generator):
    print("\n===== FINAL TEST (per-family, balanced, disjoint) =====")
    fams = list(cache.files.keys())
    all_scores, all_labels = [], []
    for fam in fams:
        Xf, yf = _balanced_block(cache, fam, rng, part="test", cap=TEST_PER_CLASS_CAP)
        if Xf.size == 0:
            print(f"  🧪 {fam:<12} | not enough samples; skip"); continue
        p_norm = odin_pnorm_numpy(Xf, encoder, clf, odin, steps=VAL_PGD_STEPS)
        scores = 1.0 - p_norm
        yhat = (scores >= thr_star).astype(int)
        prec = precision_score(yf, yhat, zero_division=0)
        rec  = recall_score(yf, yhat, zero_division=0)
        acc  = accuracy_score(yf, yhat)
        f1   = f1_score(yf, yhat, zero_division=0)
        ap   = average_precision_score(yf, scores) if yf.sum() > 0 else float("nan")
        try: auc_roc = roc_auc_score(yf, scores) if (np.unique(yf).size == 2) else float("nan")
        except Exception: auc_roc = float("nan")
        print(f"  🧪 {fam:<12} | Prec={prec:.3f}  Rec={rec:.3f}  Acc={acc:.3f}  F1={f1:.3f}  AP={ap:.3f}  AUC-ROC={auc_roc:.3f}")
        all_scores.append(scores); all_labels.append(yf)

    if all_scores:
        scores = np.concatenate(all_scores); labels = np.concatenate(all_labels)
        yhat = (scores >= thr_star).astype(int)
        prec = precision_score(labels, yhat, zero_division=0)
        rec  = recall_score(labels, yhat, zero_division=0)
        acc  = accuracy_score(labels, yhat)
        f1   = f1_score(labels, yhat, zero_division=0)
        ap   = average_precision_score(labels, scores) if labels.sum() > 0 else float("nan")
        pr_p, pr_r, _ = precision_recall_curve(labels, scores)
        auc_pr = auc(pr_r, pr_p) if len(pr_r) > 1 else float("nan")
        try:
            auc_roc = roc_auc_score(labels, scores) if (np.unique(labels).size == 2) else float("nan")
        except Exception:
            auc_roc = float("nan")
        print(f"\n[OVERALL TEST] thr={thr_star:.6f} | "
              f"Prec={prec:.3f}  Rec={rec:.3f}  Acc={acc:.3f}  F1={f1:.3f}  "
              f"AP={ap:.3f}  AUC-PR={auc_pr:.3f}  AUC-ROC={auc_roc:.3f}")
    else:
        print("\n[OVERALL TEST] No eligible families/samples on test split.")



# ------------------------ Simple VAL/TEST counters (drop-in for validate_robust) ------------------------
def validate_robust(
    cache: DataCache,
    encoder, clf, odin,
    rng: np.random.Generator,print_counts=False,
    *,
    show_test: bool = True,
    cap_val:  Optional[int] = VAL_PER_CLASS_CAP,
    cap_test: Optional[int] = TEST_PER_CLASS_CAP,
):
    """
    Simple counter-only replacement to debug data usage.
    Logs how many ID(0) and OOD(1) samples are available/used per family in VAL/TEST.
    Returns a neutral 3-tuple to keep the caller happy:
        (robust_thr, pooled_thr, per_family_dict)
    where robust_thr = pooled_thr = 0.5 and per_family_dict is an empty dict.
    """

    def _summarize(part: str, cap: Optional[int]):
      if print_counts:
        fams = sorted(cache.files.keys())
        total_id = total_ood = 0
        print(f"[VAL-COUNT] {part.upper():>5}", end=" ")
        # compute totals first (with caps if provided)
        for fam in fams:
            id_cnt  = cache.count(fam, 0, part)
            ood_cnt = cache.count(fam, 1, part)
            if cap is not None:
                id_cnt  = min(id_cnt,  cap)
                ood_cnt = min(ood_cnt, cap)
            total_id  += id_cnt
            total_ood += ood_cnt
        print(f"| used ID={total_id:,}  OOD={total_ood:,}")

        # per-family lines
        for fam in fams:
            id_cnt  = cache.count(fam, 0, part)
            ood_cnt = cache.count(fam, 1, part)
            if cap is not None:
                id_cnt  = min(id_cnt,  cap)
                ood_cnt = min(ood_cnt, cap)
            print(f"  [VAL-COUNT] {fam:<12} | used ID={id_cnt:,}  OOD={ood_cnt:,}")

    # Always show VAL with its cap
    _summarize("val", cap_val)

    # Optionally show TEST too
    if show_test:
        _summarize("test", cap_test)

    # Return dummy thresholds so existing call:
    #   thr_robust, thr_pooled, _ = validate_robust(...)
    # keeps working while you focus on margin effects.
    robust_thr = 0.5
    pooled_thr = 0.5
    per_family = {}
    return robust_thr, pooled_thr, per_family



import numpy as np
import matplotlib.pyplot as plt

def plot_roc_curves(fpr_dict, tpr_dict, auc_dict, title="ROC comparison", save_path="roc.png"):
    """
    fpr_dict: dict of {name: fpr_values}
    tpr_dict: dict of {name: tpr_values}
    auc_dict: dict of {name: auc_value}
    """

    # shrink width (figsize[0]) but keep height similar
    fig, ax = plt.subplots(figsize=(5, 4))   # was (8,6) or larger

    for name in fpr_dict.keys():
        ax.plot(
            fpr_dict[name],
            tpr_dict[name],
            lw=2,
            label=f"{name}  (AUC={auc_dict[name]:.3f})"
        )

    ax.plot([0, 1], [0, 1], "k--", lw=1)

    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel("False positive rate", fontsize=12)
    ax.set_ylabel("True positive rate", fontsize=12)
    ax.set_title(title, fontsize=13)

    # force x-axis ticks at 0.0, 0.4, 0.8, 1.0
    ax.set_xticks(np.arange(0, 1.01, 0.4))

    ax.legend(loc="lower right", fontsize=11, frameon=True)

    fig.tight_layout()
    fig.savefig(save_path, dpi=200, bbox_inches="tight")
    plt.close(fig)
    print(f"[PLOT] ROC figure saved to {save_path}")




def plot_id_ood_separation(
    cache, encoder, clf, odin, rng,
    part="val", families=None, cap_per_family=5000, bins=60,
    episode=None, save_dir="plots", show=False,
):
    import os, numpy as np, matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, precision_recall_curve, auc, average_precision_score

    # helper: balanced block
    def _balanced_block(fam):
        m = min(cache.count(fam,1,part), cache.count(fam,0,part), cap_per_family)
        if m <= 0: return np.empty((0,0),np.float32), np.empty((0,),np.int64)
        Xa = cache.sample(fam,1,part,m,rng)
        Xn = cache.sample(fam,0,part,m,rng)
        X = np.vstack([Xa,Xn]).astype(np.float32, copy=False)
        y = np.concatenate([np.ones(m,np.int64), np.zeros(m,np.int64)])
        return X,y

    fams = families if families else list(cache.files.keys())
    X_pool,y_pool=[],[]
    for fam in fams:
        Xf,yf=_balanced_block(fam)
        if Xf.size>0: X_pool.append(Xf); y_pool.append(yf)
    if not X_pool:
        return {"path":None,"scores_id":[], "scores_ood":[], "roc_auc":float("nan"), "ap":float("nan")}

    Xb = np.vstack(X_pool); yb=np.concatenate(y_pool)
    p_norm = odin_pnorm_numpy(Xb, encoder, clf, odin, steps=1)
    scores = 1.0 - p_norm
    mask=np.isfinite(scores); scores,yb=scores[mask], yb[mask]

    scores_id=scores[yb==0]; scores_ood=scores[yb==1]
    fpr,tpr,_=roc_curve(yb,scores); roc_auc=auc(fpr,tpr)
    pr_p,pr_r,_=precision_recall_curve(yb,scores); ap=average_precision_score(yb,scores)

    fig=plt.figure(figsize=(6,5))
    ax1=plt.subplot(2,2,1); ax2=plt.subplot(2,2,2); ax3=plt.subplot(2,1,2)

    if scores_id.size: ax1.hist(scores_id,bins=bins,alpha=0.6,label="ID",density=True)
    if scores_ood.size: ax1.hist(scores_ood,bins=bins,alpha=0.6,label="OOD",density=True)
    ax1.set_title("Score distribution"); ax1.set_xlabel("Anomaly score"); ax1.set_ylabel("Density")
    ax1.legend()

    ax2.plot(fpr,tpr,lw=2,label=f"AUC={roc_auc:.3f}"); ax2.plot([0,1],[0,1],"k--",lw=1)
    ax2.set_title("ROC"); ax2.set_xlabel("FPR"); ax2.set_ylabel("TPR"); ax2.legend()

    ax3.plot(pr_r,pr_p,lw=2,label=f"AP={ap:.3f}")
    ax3.set_title("Precision–Recall"); ax3.set_xlabel("Recall"); ax3.set_ylabel("Precision"); ax3.legend()

    # force xticks to 0.0,0.4,0.8,1.0 everywhere
    for ax in [ax1,ax2,ax3]:
        ax.set_xlim(0,1); ax.set_xticks([0.0,0.4,0.8,1.0])

    fig.tight_layout()
    path=None
    if save_dir:
        os.makedirs(save_dir,exist_ok=True)
        fname="sep_latest.png" if episode is None else f"sep_ep{int(episode):02d}.png"
        path=os.path.join(save_dir,fname)
        fig.savefig(path,dpi=200,bbox_inches=None,pad_inches=0.05)  # no "tight"
    if show: plt.show()
    plt.close(fig)
    return {"path":path,"scores_id":scores_id,"scores_ood":scores_ood,"roc_auc":float(roc_auc),"ap":float(ap)}

def _part_counts(cache, fam: str, part: str):
    return cache.count(fam, 0, part), cache.count(fam, 1, part)  # (ID_neg, OOD_pos)




In [22]:
# ===== Per-family thresholds ==================================================
from typing import Dict, Tuple, Optional, List
import numpy as np
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score,
    average_precision_score, roc_auc_score, precision_recall_curve, auc
)

# Reuse your existing helpers if they already exist:
# _best_f1_threshold(y_true, scores)
# _best_f1_with_min_precision(y_true, scores, min_prec)

@torch.no_grad()
def _collect_scores_by_family(
    cache, encoder, clf, odin,
    rng: np.random.Generator,
    *,
    part: str,
    cap_per_class: int,
    steps: int
) -> Tuple[Dict[str, np.ndarray], Dict[str, np.ndarray], np.ndarray, np.ndarray, Dict[str, Tuple[int,int]]]:
    """
    Returns:
      y_by_fam[fam] -> (n_fam,)
      s_by_fam[fam] -> (n_fam,)  (scores; higher == more anomalous)
      y_all, s_all  -> pooled
      used_counts[fam] -> (m_id, m_ood)
    """
    encoder.eval(); clf.eval(); odin.eval()

    fams = sorted(cache.files.keys())
    y_by_fam, s_by_fam, used_counts = {}, {}, {}

    all_y, all_s = [], []

    for fam in fams:
        m = min(cache.count(fam, 1, part), cache.count(fam, 0, part), cap_per_class)
        if m <= 0:
            continue
        Xa = cache.sample(fam, 1, part, m, rng)
        Xn = cache.sample(fam, 0, part, m, rng)
        Xf = np.vstack([Xa, Xn]).astype(np.float32, copy=False)
        yf = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)])

        # ODIN scores
        # p_norm in [0,1]; anomaly score = 1 - p_norm
        xb = torch.from_numpy(Xf).to(next(clf.parameters()).device).float()
        p_norm = odin_score_eval(xb, encoder, clf, odin, steps=steps)  # [B]
        sf = (1.0 - p_norm).cpu().numpy()

        y_by_fam[fam] = yf
        s_by_fam[fam] = sf
        used_counts[fam] = (m, m)
        all_y.append(yf); all_s.append(sf)

    if not all_y:
        return {}, {}, np.empty(0, np.int64), np.empty(0, np.float32), {}

    y_all = np.concatenate(all_y); s_all = np.concatenate(all_s)
    return y_by_fam, s_by_fam, y_all, s_all, used_counts


def pick_thresholds_per_family(
    y_by_fam: Dict[str, np.ndarray],
    s_by_fam: Dict[str, np.ndarray],
    *,
    min_precision: Optional[float] = None,
    id_ceiling_pct: Optional[float] = None
) -> Dict[str, float]:
    """
    Pick a threshold per family. If min_precision is set, use the constrained
    F1 search; otherwise normal best-F1. Optionally clamp by the given percentile
    of ID scores (to avoid exploding thresholds when positives are scarce).
    """
    thr_by_fam: Dict[str, float] = {}
    for fam, y in y_by_fam.items():
        s = s_by_fam[fam]
        if y.size == 0:
            continue
        if min_precision is None:
            t, _ = _best_f1_threshold(y, s)
        else:
            t, _ = _best_f1_with_min_precision(y, s, float(min_precision))

        if id_ceiling_pct is not None:
            s_id = s[y == 0]
            if s_id.size > 0:
                ceiling = np.percentile(s_id, float(id_ceiling_pct))
                t = min(t, float(ceiling))
        thr_by_fam[fam] = float(t)
    return thr_by_fam


def _metrics_at_threshold(y_true: np.ndarray, scores: np.ndarray, thr: float):
    yhat = (scores >= thr).astype(int)
    prec = precision_score(y_true, yhat, zero_division=0)
    rec  = recall_score(y_true, yhat, zero_division=0)
    acc  = accuracy_score(y_true, yhat)
    f1   = f1_score(y_true, yhat, zero_division=0)
    try:
        auc_roc = roc_auc_score(y_true, scores) if (np.unique(y_true).size == 2) else float("nan")
    except Exception:
        auc_roc = float("nan")
    ap = average_precision_score(y_true, scores) if y_true.sum() > 0 else float("nan")
    try:
        P, R, _ = precision_recall_curve(y_true, scores)
        pr_auc = auc(R, P)
    except Exception:
        pr_auc = float("nan")
    return dict(prec=prec, rec=rec, acc=acc, f1=f1, auc_roc=auc_roc, ap=ap, pr_auc=pr_auc)


@torch.no_grad()
def log_val_stats_after_episode_pf(
    cache,
    encoder: nn.Module,
    clf: nn.Module,
    odin: ODINParams,
    rng: np.random.Generator,
    *,
    part: str = "val",
    cap_per_class: int = 25_000,            # keep your constants if different
    steps: int = 1,
    min_precision: Optional[float] = None,  # e.g., 0.70 to relax your current 0.80
    id_ceiling_pct: Optional[float] = 95.0, # clamp family-threshold to 95th pct of ID scores
    tag: str = ""
) -> Tuple[float, Dict[str,float]]:
    """
    Validation logger that:
      1) finds a *pooled* best-F1 threshold (as before),
      2) computes a *per-family* threshold dict,
      3) prints per-family metrics at *their own* thresholds,
      4) returns (pooled_thr*, thr_by_fam).
    """
    y_by_fam, s_by_fam, y_all, s_all, used_counts = _collect_scores_by_family(
        cache, encoder, clf, odin, rng, part=part, cap_per_class=cap_per_class, steps=steps
    )

    if y_all.size == 0:
        print(f"[VAL-METRICS] {part.upper()} {tag} | no data")
        return 0.5, {}

    # Pooled best-F1 (global) — unchanged behavior
    pooled_thr, _ = _best_f1_threshold(y_all, s_all)
    pooled = _metrics_at_threshold(y_all, s_all, pooled_thr)
    print(f"[VAL-METRICS] {part.upper()} {tag} | thr*={pooled_thr:.6f} | "
          f"Prec={pooled['prec']:.3f} Rec={pooled['rec']:.3f} Acc={pooled['acc']:.3f} "
          f"F1={pooled['f1']:.3f} AP={pooled['ap']:.3f} PR-AUC={pooled['pr_auc']:.3f} "
          f"AUC-ROC={pooled['auc_roc']:.3f}")

    # Per-family thresholds
    thr_by_fam = pick_thresholds_per_family(
        y_by_fam, s_by_fam, min_precision=min_precision, id_ceiling_pct=id_ceiling_pct
    )

    print("[VAL-METRICS per-family @ own thresholds]")
    for fam in sorted(thr_by_fam.keys()):
        yf = y_by_fam[fam]; sf = s_by_fam[fam]
        t  = thr_by_fam[fam]
        stats = _metrics_at_threshold(yf, sf, t)
        mi, mo = used_counts[fam]
        print(f"  📊 {fam:<20} | thr={t:.6f} | Prec={stats['prec']:.3f} Rec={stats['rec']:.3f} "
              f"Acc={stats['acc']:.3f} F1={stats['f1']:.3f} AP={stats['ap']:.3f} "
              f"PR-AUC={stats['pr_auc']:.3f} AUC-ROC={stats['auc_roc']:.3f} "
              f"(used ID={mi:,} OOD={mo:,})")

    return float(pooled_thr), thr_by_fam


@torch.no_grad()
def test_with_thresholds(
    cache,
    encoder: nn.Module,
    clf: nn.Module,
    odin: ODINParams,
    thresholds,  # float for global OR Dict[str,float] for per-family
    rng: np.random.Generator,
    *,
    part: str = "test",
    cap_per_class: int = 25_000,
    steps: int = 1
):
    """
    Final evaluation that honors either a single global threshold or a per-family mapping.
    """
    print(f"\n===== FINAL TEST ({'per-family' if isinstance(thresholds, dict) else 'global'} thresholds) =====")
    fams = sorted(cache.files.keys())
    all_scores, all_labels = [], []

    for fam in fams:
        Xf, yf = _balanced_block(cache, fam, rng, part=part, cap=cap_per_class)
        if Xf.size == 0:
            print(f"  🧪 {fam:<20} | not enough samples; skip")
            continue

        xb = torch.from_numpy(Xf.astype(np.float32, copy=False)).to(next(clf.parameters()).device)
        p_norm = odin_score_eval(xb, encoder, clf, odin, steps=steps)
        scores = (1.0 - p_norm).cpu().numpy()

        thr = thresholds.get(fam, 0.5) if isinstance(thresholds, dict) else float(thresholds)
        yhat = (scores >= thr).astype(int)

        prec = precision_score(yf, yhat, zero_division=0)
        rec  = recall_score(yf, yhat, zero_division=0)
        acc  = accuracy_score(yf, yhat)
        f1   = f1_score(yf, yhat, zero_division=0)
        ap   = average_precision_score(yf, scores) if yf.sum() > 0 else float("nan")
        try: auc_roc = roc_auc_score(yf, scores) if (np.unique(yf).size == 2) else float("nan")
        except Exception: auc_roc = float("nan")

        print(f"  🧪 {fam:<20} | Prec={prec:.3f}  Rec={rec:.3f}  Acc={acc:.3f}  "
              f"F1={f1:.3f}  AP={ap:.3f}  AUC-ROC={auc_roc:.3f}")

        all_scores.append(scores); all_labels.append(yf)

    if all_scores:
        scores = np.concatenate(all_scores); labels = np.concatenate(all_labels)
        # For the *overall* line, apply each fam’s threshold to its own scores
        if isinstance(thresholds, dict):
            # Build an overall yhat by concatenating per-family yhat’s
            yhat_list = []
            i = 0
            for fam in fams:
                Xf, yf = _balanced_block(cache, fam, rng, part=part, cap=cap_per_class)
                if Xf.size == 0: continue
                n = yf.shape[0]
                fam_scores = scores[i:i+n]
                thr = thresholds.get(fam, 0.5)
                yhat_list.append((fam_scores >= thr).astype(int))
                i += n
            yhat = np.concatenate(yhat_list)
        else:
            yhat = (scores >= float(thresholds)).astype(int)

        prec = precision_score(labels, yhat, zero_division=0)
        rec  = recall_score(labels, yhat, zero_division=0)
        acc  = accuracy_score(labels, yhat)
        f1   = f1_score(labels, yhat, zero_division=0)
        ap   = average_precision_score(labels, scores) if labels.sum() > 0 else float("nan")
        pr_p, pr_r, _ = precision_recall_curve(labels, scores)
        auc_pr = auc(pr_r, pr_p) if len(pr_r) > 1 else float("nan")
        try: auc_roc = roc_auc_score(labels, scores) if (np.unique(labels).size == 2) else float("nan")
        except Exception: auc_roc = float("nan")

        thr_str = "<per-family>" if isinstance(thresholds, dict) else f"{float(thresholds):.6f}"
        print(f"\n[OVERALL TEST] thr={thr_str} | "
              f"Prec={prec:.3f}  Rec={rec:.3f}  Acc={acc:.3f}  F1={f1:.3f}  "
              f"AP={ap:.3f}  AUC-PR={auc_pr:.3f}  AUC-ROC={auc_roc:.3f}")


In [24]:
# ===================== Per-family metrics collector =====================

import os, json, math, numpy as np
import matplotlib.pyplot as plt

# Reuse your existing helpers if already defined:
# - _balanced_block
# - _metrics_at_threshold
# - odin_pnorm_numpy
# - VAL_PGD_STEPS, VAL_PER_CLASS_CAP
# If any of those names differ in your code, keep the bodies the same and just change the names here.

def compute_val_metrics_per_family(
    cache,
    encoder,
    clf,
    odin,
    rng: np.random.Generator,
    *,
    part: str = "val",
    cap_per_class: int = VAL_PER_CLASS_CAP,
    steps: int = VAL_PGD_STEPS,
):
    """
    Non-intrusive clone of your printed validation logic that RETURNS the numbers.
    Returns:
      thr_star: float
      pooled: dict of pooled metrics
      per_family: dict[fam] -> dict(prec, rec, auc_roc, f1, ap, pr_auc, acc, n_id, n_ood)
    """
    fams = sorted(cache.files.keys())

    # Build pooled evaluation set
    X_blocks, y_blocks, used_counts = [], [], {}
    for fam in fams:
        m = min(cache.count(fam, 1, part), cache.count(fam, 0, part), cap_per_class)
        if m <= 0:
            continue
        Xa = cache.sample(fam, 1, part, m, rng)
        Xn = cache.sample(fam, 0, part, m, rng)
        Xf = np.vstack([Xa, Xn]).astype(np.float32, copy=False)
        yf = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)])
        X_blocks.append(Xf); y_blocks.append(yf)
        used_counts[fam] = (m, m)

    if not X_blocks:
        return None, None, {}

    X_all = np.vstack(X_blocks)
    y_all = np.concatenate(y_blocks)

    # ODIN normal prob → anomaly score
    p_norm = odin_pnorm_numpy(X_all, encoder, clf, odin, steps=steps)
    scores = 1.0 - p_norm

    # Same pooled best-F1 threshold as your logger
    thr_star, _ = _best_f1_threshold(y_all, scores)
    pooled = _metrics_at_threshold(y_all, scores, thr_star)

    # Per-family stats @ global thr*
    per_family = {}
    off = 0
    for fam in fams:
        if fam not in used_counts:
            continue
        m_id, m_ood = used_counts[fam]
        n = m_id + m_ood
        yf = y_all[off:off+n]
        sf = scores[off:off+n]
        off += n
        stats = _metrics_at_threshold(yf, sf, thr_star)
        per_family[fam] = dict(
            prec=float(stats["prec"]), rec=float(stats["rec"]), auc_roc=float(stats["auc_roc"]),
            f1=float(stats["f1"]), ap=float(stats["ap"]), pr_auc=float(stats["pr_auc"]),
            acc=float(stats["acc"]), n_id=int(m_id), n_ood=int(m_ood),
        )
    return float(thr_star), pooled, per_family


class PerFamMetricHistory:
    """
    Holds metric curves over episodes. Non-invasive: you create one and call .update(...)
    after each episode. Save to disk optionally.
    """
    def __init__(self, fam_names=None):
        self.history = {}
        if fam_names is not None:
            for f in fam_names:
                self._ensure_family(f)

    def _ensure_family(self, fam):
        if fam not in self.history:
            self.history[fam] = {
                "episode": [],
                "prec": [],
                "rec": [],
                "auc_roc": [],
            }

    def update(self, episode_idx: int, per_family_stats: dict):
        # per_family_stats: {fam: {prec, rec, auc_roc, ...}}
        for fam, st in per_family_stats.items():
            self._ensure_family(fam)
            self.history[fam]["episode"].append(int(episode_idx))
            self.history[fam]["prec"].append(float(st.get("prec", float("nan"))))
            self.history[fam]["rec"].append(float(st.get("rec", float("nan"))))
            self.history[fam]["auc_roc"].append(float(st.get("auc_roc", float("nan"))))

    def to_json(self):
        return json.dumps(self.history, indent=2)

    def save_json(self, path: str):
        os.makedirs(os.path.dirname(path) or ".", exist_ok=True)
        with open(path, "w", encoding="utf-8") as f:
            json.dump(self.history, f, indent=2)

    @classmethod
    def load_json(cls, path: str):
        obj = cls()
        with open(path, "r", encoding="utf-8") as f:
            obj.history = json.load(f)
        return obj


def _slugify(text: str) -> str:
    # File-name friendly
    return "".join(ch if ch.isalnum() else "_" for ch in text).strip("_")[:140]


def plot_per_family_metric_curves(history: PerFamMetricHistory, out_dir: str = "plots_perfam_curves"):
    """
    Produces one figure per family with 3 curves: Precision, Recall, AUC-ROC vs episodes.
    """
    os.makedirs(out_dir, exist_ok=True)
    for fam, recs in history.history.items():
        if not recs["episode"]:
            continue
        xs = recs["episode"]
        ys_prec = recs["prec"]
        ys_rec  = recs["rec"]
        ys_auc  = recs["auc_roc"]

        plt.figure(figsize=(6, 4))
        plt.plot(xs, ys_prec, label="Precision")
        plt.plot(xs, ys_rec,  label="Recall")
        plt.plot(xs, ys_auc,  label="AUC-ROC")
        plt.ylim(-0.05, 1.05)
        plt.xlabel("Episode")
        plt.ylabel("Score")
        plt.title(f"{fam} — Precision / Recall / AUC-ROC vs Episode")
        plt.grid(True, linestyle="--", alpha=0.3)
        plt.legend()
        fname = os.path.join(out_dir, f"{_slugify(fam)}.png")
        plt.tight_layout()
        plt.savefig(fname, dpi=150)
        plt.close()


In [25]:
from collections import defaultdict
from dataclasses import dataclass
import numpy as np
import torch

# ---------- A) Fixed global threshold at a target FPR ----------

def _extract_labels_scores_from_valdump(val_dump):
    """
    Make this tolerant to your current validate_robust() payload.
    It tries the common shapes you've been using.
    Returns y (0=OOD, 1=ID), scores (higher => more normal).
    """
    if isinstance(val_dump, dict):
        if "y_true" in val_dump and "scores" in val_dump:
            y = np.asarray(val_dump["y_true"])
            s = np.asarray(val_dump["scores"])
            return y, s
        if "ys" in val_dump and "scores" in val_dump:
            y = np.asarray(val_dump["ys"])
            s = np.asarray(val_dump["scores"])
            return y, s
        if "id_scores" in val_dump and "ood_scores" in val_dump:
            id_s  = np.asarray(val_dump["id_scores"])
            ood_s = np.asarray(val_dump["ood_scores"])
            y = np.concatenate([np.ones_like(id_s), np.zeros_like(ood_s)])
            s = np.concatenate([id_s, ood_s])
            return y, s
    raise ValueError("Unrecognized val_dump structure; expose y_true/scores or id_scores/ood_scores.")

def compute_thr_at_fixed_fpr(val_dump, target_fpr=0.01):
    """
    Choose threshold 'thr' s.t. FPR (OOD predicted as ID) <= target_fpr,
    maximizing recall under that constraint (i.e., largest threshold that keeps FPR<=target).
    Assumes higher score => more normal (ID).
    """
    y, s = _extract_labels_scores_from_valdump(val_dump)  # y: 1=ID, 0=OOD
    neg_mask = (y == 0)  # OOD
    if neg_mask.sum() == 0:
        return float("+inf")  # degenerate; predict none as ID

    order = np.argsort(s)[::-1]  # high->low
    s_sorted = s[order]
    neg_sorted = neg_mask[order].astype(np.int32)

    cum_neg = np.cumsum(neg_sorted)  # #OOD above each cutoff
    fpr = cum_neg / max(1, neg_mask.sum())

    # indices where fpr <= target; pick the *last* (lowest thr) to maximize recall
    ok = np.where(fpr <= target_fpr)[0]
    if len(ok) == 0:
        # No threshold can satisfy target FPR; return +inf (classify none as ID)
        return float("+inf")
    k = ok[-1]
    thr = s_sorted[k]  # score >= thr => predict ID
    return float(thr)

# ---------- B) Coverage scheduler (keeps families circulating) ----------

@dataclass
class FamilyCoverageScheduler:
    all_fams: list
    heldout_fams: set

    def __post_init__(self):
        self.heldout_fams = set(self.heldout_fams)
        self.outer_counts = defaultdict(int)  # times used as OOD in outer meta loss
        self.inner_counts = defaultdict(int)  # times included in inner clean-ID push-up

    def pick_for_inner(self, allowed_fams, inner_min_fams, inner_max_fams, rng):
        """Prioritize families with lowest inner_counts."""
        cand = [f for f in allowed_fams if f not in self.heldout_fams]
        if not cand:
            return []
        cand.sort(key=lambda f: (self.inner_counts[f], rng.random()))
        k = max(inner_min_fams, min(inner_max_fams, len(cand)))
        chosen = cand[:k]
        for f in chosen:
            self.inner_counts[f] += 1
        return chosen

    def pick_for_outer(self, allowed_fams, k, rng, priority=None):
        """
        Prioritize (1) explicit priority list, then (2) lowest outer_counts.
        """
        allowed = [f for f in allowed_fams if f not in self.heldout_fams]
        if not allowed or k <= 0:
            return []
        priority = priority or []
        prio_set = set(priority)

        def keyfunc(f):
            # priority families get rank 0, others 1; then by count, then by random tiebreaker
            return (0 if f in prio_set else 1, self.outer_counts[f], rng.random())

        allowed.sort(key=keyfunc)
        chosen = allowed[:k]
        for f in chosen:
            self.outer_counts[f] += 1
        return chosen

# ---------- C) Small helper to print metrics at a fixed thr using your existing validation ----------
def print_val_metrics_at_fixed_thr(val_dump, thr, tag="(FIXED-THR)"):
    y, s = _extract_labels_scores_from_valdump(val_dump)
    y = (y.astype(int))
    pred = (s >= thr).astype(int)  # 1=ID, 0=OOD

    # basic metrics (global)
    tp = int(((pred == 1) & (y == 1)).sum())
    fp = int(((pred == 1) & (y == 0)).sum())
    tn = int(((pred == 0) & (y == 0)).sum())
    fn = int(((pred == 0) & (y == 1)).sum())

    prec = tp / max(1, (tp + fp))
    rec  = tp / max(1, (tp + fn))
    acc  = (tp + tn) / max(1, len(y))
    f1   = (2 * prec * rec) / max(1e-8, (prec + rec))

    # FPR for reference (based on OOD)
    ood_ct = max(1, (y == 0).sum())
    fpr = fp / ood_ct

    print(f"[VAL-METRICS] {tag} | thr={thr:.6f} | Prec={prec:.3f} Rec={rec:.3f} Acc={acc:.3f} F1={f1:.3f} FPR={fpr:.3f}")


In [26]:
#

In [33]:
# ------------------------ Main (minimal; fixed-FPR operating point) ------------------------
def main(
    learn_odin: bool = True,          # False => fixed (baseline); True => learn T, eps
    baseline_T: float = 1.0,
    baseline_eps: float = 1.5e-3,
    fixed_fpr: float = 0.01,          # target FPR used to set the global thr
    val_rng_seed: int = 123456,       # SAME validation pool every episode
    fix_thr_on_ep1: bool = False      # True => freeze threshold picked on EP1
):
    # ==== knobs (kept as before) ====
    SAME_EVAL_MODE   = False
    EVAL_PART        = "train" if SAME_EVAL_MODE else "val"
    CAP_PER_FAMILY   = 4000
    OVERLAY_BINS     = 60
    SEED             = 40

    set_seed(SEED)

    # ---- Discover and prepare (unchanged) ----
    DATA_DIR = "/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/"
    files = discover_files_by_category(DATA_DIR)
    if not files:
        raise RuntimeError(f"No '*.csv' found in {DATA_DIR}")

    fams_all = sorted(files.keys())
    print("Discovered families (category::subcategory):")
    for k in fams_all: print("  ", k)

    HELDOUT_FAMILIES = ['Brute Force::Web','Brute Force::XSS','DDOS attack::HOIC','Infilteration']
    heldout_resolved, missing = resolve_heldout(HELDOUT_FAMILIES, fams_all)
    if missing:
        print("\nCould not resolve some HELDOUT_FAMILIES exactly:")
        for orig, hints in missing:
            print(f"  - '{orig}' -> {('did you mean: ' + str(hints)) if hints else 'no close match found'}")

    heldout_fams    = heldout_resolved
    seen_train_fams = [f for f in fams_all if f not in heldout_fams]

    print("\nHeld-out (eval-only):", heldout_fams)
    print("Seen train families :", seen_train_fams)

    # ---- Build cache etc. (unchanged) ----
    rng   = np.random.default_rng(42)
    cache = DataCache(files, max_rows=None)
    cache.load_all()
    cache.build_splits(rng, train_frac=2/3, val_frac=1/6, test_frac=1/6)
    cache.fit_scaler_on_train(rng, use_robust=True)
    cache.apply_scaler_all()

    # ----- models (unchanged) -----
    D = cache.X[next(iter(cache.X))].shape[1]
    encoder = Encoder(in_dim=D, hid=ENC_HIDDEN, depth=ENC_DEPTH, p_drop=DROPOUT,
                      bottleneck=EMBED_DIM, feat_drop=0.05).to(device)
    decoder = Decoder(bottleneck=EMBED_DIM, hid=DEC_HIDDEN, out_dim=D, depth=DEC_DEPTH).to(device)
    clf     = Classifier(bottleneck=EMBED_DIM, num_classes=2).to(device)

    # ----- ODIN: fixed baseline vs learned (only this block differs) -----
    if learn_odin:
        odin = ODINParams(
            T0=TEMP_INIT, eps0=EPS_INIT,
            learn_T=TUNE_TEMPERATURE, learn_eps=TUNE_TEMPERATURE,
            T_bounds=(T_MIN, T_MAX), eps_bounds=(EPS_MIN, EPS_MAX)
        ).to(device)
        lr_odin_cfg   = 1e-3
        save_dir_root = "plots_perfam"
        mode_blurb    = "LEARNED-ODIN(T,eps)"
    else:
        odin = ODINParams(
            T0=baseline_T, eps0=baseline_eps,
            learn_T=False, learn_eps=False,
            T_bounds=(T_MIN, T_MAX), eps_bounds=(EPS_MIN, EPS_MAX)
        ).to(device)
        # make sure grads are off in the baseline
        for p in odin.parameters(): p.requires_grad = False
        lr_odin_cfg   = 0.0
        save_dir_root = "plots_perfam_fixed_odin"
        mode_blurb    = f"FIXED-ODIN(T={baseline_T}, eps={baseline_eps})"

    # ----- trainers (unchanged hyperparams, only lr_odin differs) -----
    steps_per_epoch_est_inner = 100
    total_inner_steps_est = N_EPISODES * INNER_EPOCHS * steps_per_epoch_est_inner
    inner_trainer = InnerTrainer(
        encoder, decoder, clf,
        lr_max=INNER_LR_MAX, weight_decay=WEIGHT_DECAY, clip_norm=CLIP_NORM,
        use_amp=USE_AMP, lambda_rec=LAMBDA_REC,
        total_inner_steps_est=total_inner_steps_est
    )

    steps_per_epoch_est_outer = 50
    total_outer_steps_est = N_EPISODES * OUTER_EPOCHS * steps_per_epoch_est_outer
    outer_trainer = OuterTrainer(
        encoder, clf, odin,
        lr_odin=lr_odin_cfg, lr_phi=3e-3,
        wd_odin=0.0, wd_phi=1e-4,
        total_outer_steps_est=total_outer_steps_est,
        unfreeze_encoder=False,
        enc_lr=1e-4, enc_wd=1e-4,
        logT_l2=0.0, logeps_l2=0.0,
        amp=USE_AMP,
        margin_m=0.50, margin_alpha=0.10
    )

    rng_epi = np.random.default_rng(SEED + 777)

    # fixed per-episode eval pools (unchanged)
    eval_pools = build_perfam_eval_pools(cache, fams_all, rng_epi, part=EVAL_PART, cap_per_family=CAP_PER_FAMILY)

    print(f"\n=== Running in mode: {mode_blurb} | TPR@FPR target FPR={fixed_fpr:.1%}; "
          f"val-pool fixed by seed={val_rng_seed} | thr policy={'FREEZE@EP1' if fix_thr_on_ep1 else 'RECOMPUTE/Episode'} ===")

    # ---- threshold policy state ----
    thr_fixed = None  # if fix_thr_on_ep1=True, we set this on episode 1 and reuse

    for ep in range(1, N_EPISODES + 1):
        print(f"\n=========== EPISODE {ep:02d} ({mode_blurb}) ===========")

        # ---------- PRE (unchanged) ----------
        results_pre = eval_stage_per_family(eval_pools, encoder, clf, odin, steps=1, stage_name="pre")

        # ---------- INNER (unchanged) ----------
        META_TASKS = int(rng_epi.integers(1, 3))
        meta_tasks = []
        inner_used_union: List[str] = []
        for _ in range(META_TASKS):
            inner_fams = select_inner_fams_up_to(
                cache, rng_epi, allowed_fams=seen_train_fams, part="train",
                inner_max_fams=INNER_MAX_FAMS, inner_min_fams=2
            )
            if len(inner_fams) < 2: continue
            Xi, yi, used = build_inner_episode_batch(
                cache, rng_epi, part="train", fams=inner_fams,
                pos_per_fam=INNER_POS_PER_FAM, neg_per_fam=INNER_NEG_PER_FAM
            )
            if Xi.size == 0: continue
            meta_tasks.append((Xi.astype(np.float32), yi.astype(np.int64), used))
            inner_used_union.extend(used)

        if len(meta_tasks) == 0:
            print("  (skip: no meta tasks built)")
            continue

        print(f"[EP {ep:02d}] META inner: {len(meta_tasks)} tasks; union_fams={sorted(set(inner_used_union))}")
        inner_trainer.train_one_episode(meta_tasks, batch_size=INNER_BATCH, epochs=INNER_EPOCHS)

        # ---------- INNER EVAL (unchanged) ----------
        results_inner = eval_stage_per_family(eval_pools, encoder, clf, odin, steps=1, stage_name="inner")

        # ---------- OUTER (unchanged selection) ----------
        allowed_for_outer = [f for f in seen_train_fams if (f not in heldout_fams) and (f not in inner_used_union)]
        if HARD_OOD_FAMS:
            offset = (ep - 1) % len(HARD_OOD_FAMS)
            rot = HARD_OOD_FAMS[offset:] + HARD_OOD_FAMS[:offset]
        else:
            rot = []
        priority_ood = [f for f in rot if f in allowed_for_outer]
        fill_pool = [f for f in allowed_for_outer if f not in priority_ood]
        rng_epi.shuffle(fill_pool)
        target_k = min(TARGET_OOD_K, OUTER_MAX_FAMS, len(priority_ood) + len(fill_pool))
        ood_fams = (priority_ood + fill_pool)[:target_k]

        if not ood_fams:
            print("  (skip outer: no OOD families available)")
            results_outer = {}
        else:
            Xq, yq, used_fams = build_outer_episode_batch(
                cache, rng_epi,
                part="train",
                ood_fams=ood_fams,
                fs_num_ood_classes=FS_NUM_OOD_CLASSES,
                fs_k_per_class=FS_K_PER_CLASS_OUTER,
                allow_smaller=FS_ALLOW_SMALLER,
            )
            if Xq.size == 0:
                print("  (skip outer: empty outer batch)")
                results_outer = {}
            else:
                pos_ct = int(yq.sum()); neg_ct = int(len(yq) - pos_ct)
                print(f"[EP {ep:02d}] OUTER | OOD={used_fams} | X={Xq.shape} (pos={pos_ct}, neg={neg_ct})")
                outer_trainer.train_one_episode(
                    Xq=Xq.astype(np.float32), yq=yq.astype(np.int64),
                    batch_size=OUTER_BATCH, epochs=OUTER_EPOCHS,
                    steps=TRAIN_PGD_STEPS, shuffle=True
                )
                results_outer = eval_stage_per_family(eval_pools, encoder, clf, odin, steps=1, stage_name="outer")

        # ---------- PLOTS (unchanged) ----------
        plot_per_family_overlays(
            results_pre=results_pre,
            results_inner=results_inner,
            results_outer=results_outer,
            episode=ep,
            save_root=save_dir_root,
            bins=OVERLAY_BINS,
            verbose=False
        )

        # ---------- VALIDATION @ fixed FPR on SAME pool every episode ----------
        rng_val_fixed = np.random.default_rng(val_rng_seed)
        thr_for_print = None
        try:
            # keep your existing robust validator; do NOT remove it
            thr_robust, thr_pooled, val_dump = validate_robust(
                cache, encoder, clf, odin, rng_val_fixed, print_counts=False
            )

            # policy: recompute every episode OR freeze on ep1 and reuse
            if fix_thr_on_ep1:
                if thr_fixed is None:
                    thr_fixed = compute_thr_at_fixed_fpr(val_dump, target_fpr=fixed_fpr)
                thr_for_print = thr_fixed
            else:
                thr_for_print = compute_thr_at_fixed_fpr(val_dump, target_fpr=fixed_fpr)

            # your original printed line (unchanged content/format)
            print_val_metrics_at_fixed_thr(
                val_dump, thr=thr_for_print, tag=f"VAL (EP {ep:02d}) TPR@FPR={fixed_fpr:.1%}"
            )
        except Exception as e:
            print(f"[VAL] Warning in validate_robust or thresholding: {e}")

        # --- optional: per-family logs (pass fixed_thr if your function supports it; otherwise fallback) ---
        try:
            try:
                log_val_stats_after_episode(
                    cache=cache, encoder=encoder, clf=clf, odin=odin,
                    rng=np.random.default_rng(val_rng_seed),
                    part="val" if not SAME_EVAL_MODE else "train",
                    cap_per_class=VAL_PER_CLASS_CAP, steps=VAL_PGD_STEPS,
                    tag=f"(EP {ep:02d})",
                    fixed_thr=thr_for_print
                )
            except TypeError:
                log_val_stats_after_episode(
                    cache=cache, encoder=encoder, clf=clf, odin=odin,
                    rng=np.random.default_rng(val_rng_seed),
                    part="val" if not SAME_EVAL_MODE else "train",
                    cap_per_class=VAL_PER_CLASS_CAP, steps=VAL_PGD_STEPS,
                    tag=f"(EP {ep:02d})"
                )
        except Exception as e:
            print(f"[VAL METRICS] Warning: {e}")

        # cleanup
        for (Xi, yi, _) in meta_tasks: del Xi, yi
        if 'Xq' in locals(): del Xq, yq
        gc.collect()

    print("\n===== FINAL TEST (one-shot, after all episodes) =====")
    # reuse threshold policy for final test
    try:
        rng_val_fixed = np.random.default_rng(val_rng_seed)
        _, _, val_dump = validate_robust(cache, encoder, clf, odin, rng_val_fixed, print_counts=False)
        if fix_thr_on_ep1 and (thr_fixed is not None):
            thr_final = thr_fixed
        else:
            thr_final = compute_thr_at_fixed_fpr(val_dump, target_fpr=fixed_fpr)
    except Exception:
        thr_final = 0.5

    test_per_family(cache, encoder, clf, odin, thr_star=thr_final, rng=np.random.default_rng(SEED + 888))


In [34]:
main(learn_odin=True, fixed_fpr=0.01, val_rng_seed=123456, fix_thr_on_ep1=False)

Discovered families (category::subcategory):
   Bot
   Brute Force::Web
   Brute Force::XSS
   DDOS attack::HOIC
   DDOS attack::LOIC-UDP
   DDoS attacks::LOIC-HTTP
   DoS attacks::GoldenEye
   DoS attacks::Hulk
   DoS attacks::SlowHTTPTest
   DoS attacks::Slowloris
   FTP::BruteForce
   Infilteration
   SQL Injection
   SSH::Bruteforce

Held-out (eval-only): ['Brute Force::Web', 'Brute Force::XSS', 'DDOS attack::HOIC', 'Infilteration']
Seen train families : ['Bot', 'DDOS attack::LOIC-UDP', 'DDoS attacks::LOIC-HTTP', 'DoS attacks::GoldenEye', 'DoS attacks::Hulk', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Slowloris', 'FTP::BruteForce', 'SQL Injection', 'SSH::Bruteforce']

=== Running in mode: LEARNED-ODIN(T,eps) | TPR@FPR target FPR=1.0%; val-pool fixed by seed=123456 | thr policy=RECOMPUTE/Episode ===

=========== EPISODE 01 (LEARNED-ODIN(T,eps)) ===========
[EP 01] META inner: 1 tasks; union_fams=['DDOS attack::LOIC-UDP', 'DoS attacks::GoldenEye']
[INNER/BCE] Ep01 | Loss=2.80960 | lr

/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


[PLOT] Per-family overlays saved (14 figs) → plots_perfam/ep06
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; need y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 06) | thr*=0.000001 | Prec=0.783 Rec=0.917 Acc=0.831 F1=0.844 AP=0.943 PR-AUC=0.946 AUC-ROC=0.930
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.740 Rec=0.998 Acc=0.824 F1=0.850 AP=0.649 PR-AUC=0.578 AUC-ROC=0.734  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.485 Rec=0.804 Acc=0.475 F1=0.605 AP=0.783 PR-AUC=0.773 AUC-ROC=0.732  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.514 Rec=1.000 Acc=0.526 F1=0.679 AP=0.972 PR-AUC=0.974 AUC-ROC=0.964  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.756 Rec=1.000 Acc=0.838 F1=0.861 AP=0.972 PR-AUC=0.972 AUC-ROC=0.977  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.653 Rec=1.000 Acc=0.734 F1=0.790 AP=1.000 PR-AUC=1.000 AUC-ROC=1.000  (used ID=288 OOD=288)
  📊 DDoS attacks::LOIC-HTTP |

/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges


[PLOT] Per-family overlays saved (14 figs) → plots_perfam/ep07
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; need y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 07) | thr*=0.004431 | Prec=0.962 Rec=0.770 Acc=0.869 F1=0.855 AP=0.938 PR-AUC=0.939 AUC-ROC=0.917
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.000 Rec=0.000 Acc=0.467 F1=0.000 AP=0.580 PR-AUC=0.535 AUC-ROC=0.626  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.604 Rec=0.598 Acc=0.603 F1=0.601 AP=0.742 PR-AUC=0.725 AUC-ROC=0.748  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.357 Rec=0.526 Acc=0.289 F1=0.426 AP=0.707 PR-AUC=0.700 AUC-ROC=0.511  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.987 Rec=1.000 Acc=0.993 F1=0.993 AP=0.976 PR-AUC=0.976 AUC-ROC=0.991  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.986 Rec=1.000 Acc=0.993 F1=0.993 AP=1.000 PR-AUC=1.000 AUC-ROC=1.000  (used ID=288 OOD=288)
  📊 DDoS attacks::LOIC-HTTP |

...

[PLOT] Per-family overlays saved (14 figs) → plots_perfam/ep12
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; need y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 12) | thr*=0.004431 | Prec=0.967 Rec=0.745 Acc=0.860 F1=0.842 AP=0.923 PR-AUC=0.923 AUC-ROC=0.888
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.000 Rec=0.000 Acc=0.467 F1=0.000 AP=0.516 PR-AUC=0.473 AUC-ROC=0.449  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.740 Rec=0.755 Acc=0.745 F1=0.748 AP=0.789 PR-AUC=0.779 AUC-ROC=0.801  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.357 Rec=0.526 Acc=0.289 F1=0.426 AP=0.600 PR-AUC=0.582 AUC-ROC=0.506  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.995 Rec=0.793 Acc=0.894 F1=0.882 AP=0.968 PR-AUC=0.968 AUC-ROC=0.975  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.990 Rec=1.000 Acc=0.995 F1=0.995 AP=1.000 PR-AUC=1.000 AUC-ROC=1.000  (used ID=288 OOD=288)
  📊 DDoS attacks::LOIC-HTTP |

/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges


[PLOT] Per-family overlays saved (14 figs) → plots_perfam/ep13
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; need y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 13) | thr*=0.000634 | Prec=0.956 Rec=0.770 Acc=0.867 F1=0.853 AP=0.938 PR-AUC=0.940 AUC-ROC=0.921
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.003 Rec=0.000 Acc=0.443 F1=0.001 AP=0.645 PR-AUC=0.531 AUC-ROC=0.640  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.740 Rec=0.755 Acc=0.745 F1=0.748 AP=0.736 PR-AUC=0.728 AUC-ROC=0.686  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.357 Rec=0.526 Acc=0.289 F1=0.426 AP=0.559 PR-AUC=0.542 AUC-ROC=0.487  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.993 Rec=0.983 Acc=0.988 F1=0.988 AP=0.986 PR-AUC=0.986 AUC-ROC=0.995  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.993 Rec=1.000 Acc=0.997 F1=0.997 AP=1.000 PR-AUC=1.000 AUC-ROC=1.000  (used ID=288 OOD=288)
  📊 DDoS attacks::LOIC-HTTP |

/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


[PLOT] Per-family overlays saved (14 figs) → plots_perfam/ep14
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; need y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 14) | thr*=0.000001 | Prec=0.812 Rec=0.689 Acc=0.765 F1=0.746 AP=0.826 PR-AUC=0.855 AUC-ROC=0.799
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.002 Rec=0.000 Acc=0.391 F1=0.001 AP=0.500 PR-AUC=0.251 AUC-ROC=0.350  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.534 Rec=0.765 Acc=0.549 F1=0.629 AP=0.726 PR-AUC=0.719 AUC-ROC=0.700  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.507 Rec=1.000 Acc=0.513 F1=0.673 AP=0.549 PR-AUC=0.528 AUC-ROC=0.490  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.778 Rec=0.212 Acc=0.576 F1=0.333 AP=0.582 PR-AUC=0.635 AUC-ROC=0.559  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.718 Rec=1.000 Acc=0.804 F1=0.836 AP=1.000 PR-AUC=1.000 AUC-ROC=1.000  (used ID=288 OOD=288)
  📊 DDoS attacks::LOIC-HTTP |

In [35]:
main(learn_odin=False, baseline_T=1.0, baseline_eps=1.5e-3, fixed_fpr=0.01, val_rng_seed=123456, fix_thr_on_ep1=False)

Discovered families (category::subcategory):
   Bot
   Brute Force::Web
   Brute Force::XSS
   DDOS attack::HOIC
   DDOS attack::LOIC-UDP
   DDoS attacks::LOIC-HTTP
   DoS attacks::GoldenEye
   DoS attacks::Hulk
   DoS attacks::SlowHTTPTest
   DoS attacks::Slowloris
   FTP::BruteForce
   Infilteration
   SQL Injection
   SSH::Bruteforce

Held-out (eval-only): ['Brute Force::Web', 'Brute Force::XSS', 'DDOS attack::HOIC', 'Infilteration']
Seen train families : ['Bot', 'DDOS attack::LOIC-UDP', 'DDoS attacks::LOIC-HTTP', 'DoS attacks::GoldenEye', 'DoS attacks::Hulk', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Slowloris', 'FTP::BruteForce', 'SQL Injection', 'SSH::Bruteforce']

=== Running in mode: FIXED-ODIN(T=1.0, eps=0.0015) | TPR@FPR target FPR=1.0%; val-pool fixed by seed=123456 | thr policy=RECOMPUTE/Episode ===

=========== EPISODE 01 (FIXED-ODIN(T=1.0, eps=0.0015)) ===========
[EP 01] META inner: 1 tasks; union_fams=['DDOS attack::LOIC-UDP', 'DoS attacks::GoldenEye']
[INNER/BCE] Ep01

/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


[PLOT] Per-family overlays saved (14 figs) → plots_perfam_fixed_odin/ep06
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; need y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 06) | thr*=0.000001 | Prec=0.848 Rec=0.907 Acc=0.872 F1=0.876 AP=0.942 PR-AUC=0.947 AUC-ROC=0.925
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.777 Rec=0.999 Acc=0.856 F1=0.874 AP=0.666 PR-AUC=0.633 AUC-ROC=0.789  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.628 Rec=0.961 Acc=0.696 F1=0.760 AP=0.819 PR-AUC=0.810 AUC-ROC=0.857  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.500 Rec=1.000 Acc=0.500 F1=0.667 AP=0.972 PR-AUC=0.973 AUC-ROC=0.962  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.782 Rec=0.989 Acc=0.856 F1=0.873 AP=0.947 PR-AUC=0.947 AUC-ROC=0.978  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.709 Rec=1.000 Acc=0.795 F1=0.830 AP=0.999 PR-AUC=0.999 AUC-ROC=0.999  (used ID=288 OOD=288)
  📊 DDoS attacks::

/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


[PLOT] Per-family overlays saved (14 figs) → plots_perfam_fixed_odin/ep07
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; need y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 07) | thr*=0.000634 | Prec=0.932 Rec=0.821 Acc=0.881 F1=0.873 AP=0.945 PR-AUC=0.945 AUC-ROC=0.920
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.727 Rec=0.492 Acc=0.654 F1=0.587 AP=0.619 PR-AUC=0.618 AUC-ROC=0.760  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.681 Rec=0.961 Acc=0.755 F1=0.797 AP=0.805 PR-AUC=0.804 AUC-ROC=0.825  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.357 Rec=0.526 Acc=0.289 F1=0.426 AP=0.712 PR-AUC=0.708 AUC-ROC=0.524  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.914 Rec=1.000 Acc=0.953 F1=0.955 AP=0.987 PR-AUC=0.987 AUC-ROC=0.993  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.847 Rec=1.000 Acc=0.910 F1=0.917 AP=1.000 PR-AUC=1.000 AUC-ROC=1.000  (used ID=288 OOD=288)
  📊 DDoS attacks::

...

[PLOT] Per-family overlays saved (14 figs) → plots_perfam_fixed_odin/ep12
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; need y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 12) | thr*=0.001900 | Prec=0.959 Rec=0.811 Acc=0.888 F1=0.879 AP=0.947 PR-AUC=0.947 AUC-ROC=0.926
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.818 Rec=0.452 Acc=0.676 F1=0.582 AP=0.714 PR-AUC=0.713 AUC-ROC=0.847  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.642 Rec=0.755 Acc=0.667 F1=0.694 AP=0.762 PR-AUC=0.752 AUC-ROC=0.783  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.357 Rec=0.526 Acc=0.289 F1=0.426 AP=0.519 PR-AUC=0.509 AUC-ROC=0.442  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.995 Rec=0.972 Acc=0.984 F1=0.984 AP=0.996 PR-AUC=0.996 AUC-ROC=0.998  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.986 Rec=1.000 Acc=0.993 F1=0.993 AP=1.000 PR-AUC=1.000 AUC-ROC=1.000  (used ID=288 OOD=288)
  📊 DDoS attacks::

/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


[PLOT] Per-family overlays saved (14 figs) → plots_perfam_fixed_odin/ep13
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; need y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 13) | thr*=0.000634 | Prec=0.964 Rec=0.758 Acc=0.865 F1=0.849 AP=0.931 PR-AUC=0.932 AUC-ROC=0.906
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.107 Rec=0.016 Acc=0.442 F1=0.027 AP=0.555 PR-AUC=0.521 AUC-ROC=0.601  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.750 Rec=0.765 Acc=0.755 F1=0.757 AP=0.885 PR-AUC=0.880 AUC-ROC=0.869  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.385 Rec=0.526 Acc=0.342 F1=0.444 AP=0.526 PR-AUC=0.513 AUC-ROC=0.496  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.962 Rec=0.971 Acc=0.966 F1=0.966 AP=0.937 PR-AUC=0.937 AUC-ROC=0.971  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.993 Rec=1.000 Acc=0.997 F1=0.997 AP=1.000 PR-AUC=1.000 AUC-ROC=1.000  (used ID=288 OOD=288)
  📊 DDoS attacks::

/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


[PLOT] Per-family overlays saved (14 figs) → plots_perfam_fixed_odin/ep14
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; need y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 14) | thr*=0.000001 | Prec=0.713 Rec=0.935 Acc=0.779 F1=0.809 AP=0.903 PR-AUC=0.904 AUC-ROC=0.885
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.695 Rec=0.996 Acc=0.780 F1=0.819 AP=0.661 PR-AUC=0.633 AUC-ROC=0.778  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.598 Rec=0.961 Acc=0.657 F1=0.737 AP=0.750 PR-AUC=0.741 AUC-ROC=0.790  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.507 Rec=1.000 Acc=0.513 F1=0.673 AP=0.578 PR-AUC=0.569 AUC-ROC=0.565  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.798 Rec=1.000 Acc=0.873 F1=0.887 AP=0.838 PR-AUC=0.742 AUC-ROC=0.819  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.935 Rec=1.000 Acc=0.965 F1=0.966 AP=1.000 PR-AUC=1.000 AUC-ROC=1.000  (used ID=288 OOD=288)
  📊 DDoS attacks::

In [ ]:
# if __name__ == "__main__":
#     main()

main(learn_odin=False, baseline_T=1.0, baseline_eps=1.5e-3) ### Fixed-(T,ε) baseline

Discovered families (category::subcategory):
   Bot
   Brute Force::Web
   Brute Force::XSS
   DDOS attack::HOIC
   DDOS attack::LOIC-UDP
   DDoS attacks::LOIC-HTTP
   DoS attacks::GoldenEye
   DoS attacks::Hulk
   DoS attacks::SlowHTTPTest
   DoS attacks::Slowloris
   FTP::BruteForce
   Infilteration
   SQL Injection
   SSH::Bruteforce

Held-out (eval-only): ['Brute Force::Web', 'Brute Force::XSS', 'DDOS attack::HOIC', 'Infilteration']
Seen train families : ['Bot', 'DDOS attack::LOIC-UDP', 'DDoS attacks::LOIC-HTTP', 'DoS attacks::GoldenEye', 'DoS attacks::Hulk', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Slowloris', 'FTP::BruteForce', 'SQL Injection', 'SSH::Bruteforce']

=== Running in mode: FIXED-ODIN(T=1.0, eps=0.0015) | TPR@FPR target FPR=1.0%; val-pool fixed by seed=123456 ===

=========== EPISODE 01 (FIXED-ODIN(T=1.0, eps=0.0015)) ===========
[EP 01] META inner: 1 tasks; union_fams=['DDOS attack::LOIC-UDP', 'DoS attacks::GoldenEye']
[INNER/BCE] Ep01 | Loss=2.80960 | lr=3.00e-03
[

/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


[PLOT] Per-family overlays saved (14 figs) → plots_perfam_fixed_odin/ep06
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; expose y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 06) | thr*=0.000001 | Prec=0.848 Rec=0.907 Acc=0.872 F1=0.876 AP=0.942 PR-AUC=0.947 AUC-ROC=0.925
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.777 Rec=0.999 Acc=0.856 F1=0.874 AP=0.666 PR-AUC=0.633 AUC-ROC=0.789  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.628 Rec=0.961 Acc=0.696 F1=0.760 AP=0.819 PR-AUC=0.810 AUC-ROC=0.857  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.500 Rec=1.000 Acc=0.500 F1=0.667 AP=0.972 PR-AUC=0.973 AUC-ROC=0.962  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.782 Rec=0.989 Acc=0.856 F1=0.873 AP=0.947 PR-AUC=0.947 AUC-ROC=0.978  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.709 Rec=1.000 Acc=0.795 F1=0.830 AP=0.999 PR-AUC=0.999 AUC-ROC=0.999  (used ID=288 OOD=288)
  📊 DDoS attacks

/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


[PLOT] Per-family overlays saved (14 figs) → plots_perfam_fixed_odin/ep07
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; expose y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 07) | thr*=0.000634 | Prec=0.932 Rec=0.821 Acc=0.881 F1=0.873 AP=0.945 PR-AUC=0.945 AUC-ROC=0.920
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.727 Rec=0.492 Acc=0.654 F1=0.587 AP=0.619 PR-AUC=0.618 AUC-ROC=0.760  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.681 Rec=0.961 Acc=0.755 F1=0.797 AP=0.805 PR-AUC=0.804 AUC-ROC=0.825  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.357 Rec=0.526 Acc=0.289 F1=0.426 AP=0.712 PR-AUC=0.708 AUC-ROC=0.524  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.914 Rec=1.000 Acc=0.953 F1=0.955 AP=0.987 PR-AUC=0.987 AUC-ROC=0.993  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.847 Rec=1.000 Acc=0.910 F1=0.917 AP=1.000 PR-AUC=1.000 AUC-ROC=1.000  (used ID=288 OOD=288)
  📊 DDoS attacks

...

[PLOT] Per-family overlays saved (14 figs) → plots_perfam_fixed_odin/ep12
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; expose y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 12) | thr*=0.001900 | Prec=0.959 Rec=0.811 Acc=0.888 F1=0.879 AP=0.947 PR-AUC=0.947 AUC-ROC=0.926
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.818 Rec=0.452 Acc=0.676 F1=0.582 AP=0.714 PR-AUC=0.713 AUC-ROC=0.847  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.642 Rec=0.755 Acc=0.667 F1=0.694 AP=0.762 PR-AUC=0.752 AUC-ROC=0.783  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.357 Rec=0.526 Acc=0.289 F1=0.426 AP=0.519 PR-AUC=0.509 AUC-ROC=0.442  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.995 Rec=0.972 Acc=0.984 F1=0.984 AP=0.996 PR-AUC=0.996 AUC-ROC=0.998  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.986 Rec=1.000 Acc=0.993 F1=0.993 AP=1.000 PR-AUC=1.000 AUC-ROC=1.000  (used ID=288 OOD=288)
  📊 DDoS attacks

/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


[PLOT] Per-family overlays saved (14 figs) → plots_perfam_fixed_odin/ep13
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; expose y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 13) | thr*=0.000634 | Prec=0.964 Rec=0.758 Acc=0.865 F1=0.849 AP=0.931 PR-AUC=0.932 AUC-ROC=0.906
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.107 Rec=0.016 Acc=0.442 F1=0.027 AP=0.555 PR-AUC=0.521 AUC-ROC=0.601  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.750 Rec=0.765 Acc=0.755 F1=0.757 AP=0.885 PR-AUC=0.880 AUC-ROC=0.869  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.385 Rec=0.526 Acc=0.342 F1=0.444 AP=0.526 PR-AUC=0.513 AUC-ROC=0.496  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.962 Rec=0.971 Acc=0.966 F1=0.966 AP=0.937 PR-AUC=0.937 AUC-ROC=0.971  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.993 Rec=1.000 Acc=0.997 F1=0.997 AP=1.000 PR-AUC=1.000 AUC-ROC=1.000  (used ID=288 OOD=288)
  📊 DDoS attacks

/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


[PLOT] Per-family overlays saved (14 figs) → plots_perfam_fixed_odin/ep14
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; expose y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 14) | thr*=0.000001 | Prec=0.713 Rec=0.935 Acc=0.779 F1=0.809 AP=0.903 PR-AUC=0.904 AUC-ROC=0.885
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.695 Rec=0.996 Acc=0.780 F1=0.819 AP=0.661 PR-AUC=0.633 AUC-ROC=0.778  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.598 Rec=0.961 Acc=0.657 F1=0.737 AP=0.750 PR-AUC=0.741 AUC-ROC=0.790  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.507 Rec=1.000 Acc=0.513 F1=0.673 AP=0.578 PR-AUC=0.569 AUC-ROC=0.565  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.798 Rec=1.000 Acc=0.873 F1=0.887 AP=0.838 PR-AUC=0.742 AUC-ROC=0.819  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.935 Rec=1.000 Acc=0.965 F1=0.966 AP=1.000 PR-AUC=1.000 AUC-ROC=1.000  (used ID=288 OOD=288)
  📊 DDoS attacks

In [ ]:
if __name__ == "__main__":
    main()

Discovered families (category::subcategory):
   Bot
   Brute Force::Web
   Brute Force::XSS
   DDOS attack::HOIC
   DDOS attack::LOIC-UDP
   DDoS attacks::LOIC-HTTP
   DoS attacks::GoldenEye
   DoS attacks::Hulk
   DoS attacks::SlowHTTPTest
   DoS attacks::Slowloris
   FTP::BruteForce
   Infilteration
   SQL Injection
   SSH::Bruteforce

Held-out (eval-only): ['Brute Force::Web', 'Brute Force::XSS', 'DDOS attack::HOIC', 'Infilteration']
Seen train families : ['Bot', 'DDOS attack::LOIC-UDP', 'DDoS attacks::LOIC-HTTP', 'DoS attacks::GoldenEye', 'DoS attacks::Hulk', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Slowloris', 'FTP::BruteForce', 'SQL Injection', 'SSH::Bruteforce']

=== Running in mode: LEARNED-ODIN(T,eps) | TPR@FPR target FPR=1.0%; val-pool fixed by seed=123456 ===

=========== EPISODE 01 (LEARNED-ODIN(T,eps)) ===========
[EP 01] META inner: 1 tasks; union_fams=['DDOS attack::LOIC-UDP', 'DoS attacks::GoldenEye']
[INNER/BCE] Ep01 | Loss=2.23338 | lr=3.00e-03
[INNER/BCE] Ep02 | Lo

/usr/local/lib/python3.12/dist-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


[PLOT] Per-family overlays saved (14 figs) → plots_perfam/ep07
[VAL] Warning in validate_robust or thresholding: Unrecognized val_dump structure; expose y_true/scores or id_scores/ood_scores.
[VAL-METRICS] VAL (EP 07) | thr*=0.009494 | Prec=0.890 Rec=0.834 Acc=0.866 F1=0.861 AP=0.939 PR-AUC=0.939 AUC-ROC=0.916
[VAL-METRICS per-family @ global thr*]
  📊 Bot          | Prec=0.673 Rec=0.492 Acc=0.626 F1=0.568 AP=0.568 PR-AUC=0.567 AUC-ROC=0.704  (used ID=25,000 OOD=25,000)
  📊 Brute Force::Web | Prec=0.748 Rec=0.961 Acc=0.819 F1=0.841 AP=0.818 PR-AUC=0.817 AUC-ROC=0.830  (used ID=102 OOD=102)
  📊 Brute Force::XSS | Prec=0.357 Rec=0.526 Acc=0.289 F1=0.426 AP=0.538 PR-AUC=0.524 AUC-ROC=0.445  (used ID=38 OOD=38)
  📊 DDOS attack::HOIC | Prec=0.849 Rec=1.000 Acc=0.911 F1=0.919 AP=0.913 PR-AUC=0.913 AUC-ROC=0.969  (used ID=25,000 OOD=25,000)
  📊 DDOS attack::LOIC-UDP | Prec=0.986 Rec=1.000 Acc=0.993 F1=0.993 AP=1.000 PR-AUC=1.000 AUC-ROC=1.000  (used ID=288 OOD=288)
  📊 DDoS attacks::LOIC-HTTP

In [ ]:
import re
import os
import matplotlib.pyplot as plt
import numpy as np

# === Directory to save plots ===
SAVE_DIR = "./val_metric_plots_run12_no_learn"
os.makedirs(SAVE_DIR, exist_ok=True)

# === Paste your log data here ===
LOGS = r"""

[VAL-METRICS] VAL (EP 01) | thr*=0.000001 | Prec=0.500 Rec=1.000 Acc=0.500 F1=0.667 AP=0.590 PR-AUC=0.588 AUC-ROC=0.465
[VAL-METRICS] VAL (EP 02) | thr*=0.000634 | Prec=0.746 Rec=0.666 Acc=0.720 F1=0.704 AP=0.818 PR-AUC=0.816 AUC-ROC=0.779
[VAL-METRICS] VAL (EP 03) | thr*=0.001900 | Prec=0.924 Rec=0.618 Acc=0.784 F1=0.741 AP=0.914 PR-AUC=0.914 AUC-ROC=0.898
[VAL-METRICS] VAL (EP 04) | thr*=0.000634 | Prec=0.887 Rec=0.568 Acc=0.748 F1=0.692 AP=0.886 PR-AUC=0.886 AUC-ROC=0.870
[VAL-METRICS] VAL (EP 05) | thr*=0.001267 | Prec=0.937 Rec=0.754 Acc=0.852 F1=0.836 AP=0.924 PR-AUC=0.924 AUC-ROC=0.897
[VAL-METRICS] VAL (EP 06) | thr*=0.000001 | Prec=0.848 Rec=0.907 Acc=0.872 F1=0.876 AP=0.942 PR-AUC=0.947 AUC-ROC=0.925
[VAL-METRICS] VAL (EP 07) | thr*=0.000634 | Prec=0.932 Rec=0.821 Acc=0.881 F1=0.873 AP=0.945 PR-AUC=0.945 AUC-ROC=0.920
[VAL-METRICS] VAL (EP 08) | thr*=0.000634 | Prec=0.966 Rec=0.752 Acc=0.863 F1=0.846 AP=0.943 PR-AUC=0.943 AUC-ROC=0.922
[VAL-METRICS] VAL (EP 09) | thr*=0.001900 | Prec=0.972 Rec=0.755 Acc=0.867 F1=0.850 AP=0.947 PR-AUC=0.948 AUC-ROC=0.928
[VAL-METRICS] VAL (EP 10) | thr*=0.004431 | Prec=0.971 Rec=0.755 Acc=0.866 F1=0.849 AP=0.944 PR-AUC=0.944 AUC-ROC=0.924
[VAL-METRICS] VAL (EP 11) | thr*=0.000001 | Prec=0.865 Rec=0.828 Acc=0.849 F1=0.846 AP=0.933 PR-AUC=0.937 AUC-ROC=0.915
[VAL-METRICS] VAL (EP 12) | thr*=0.001900 | Prec=0.959 Rec=0.811 Acc=0.888 F1=0.879 AP=0.947 PR-AUC=0.947 AUC-ROC=0.926
[VAL-METRICS] VAL (EP 13) | thr*=0.000634 | Prec=0.964 Rec=0.758 Acc=0.865 F1=0.849 AP=0.931 PR-AUC=0.932 AUC-ROC=0.906
[VAL-METRICS] VAL (EP 14) | thr*=0.000001 | Prec=0.713 Rec=0.935 Acc=0.779 F1=0.809 AP=0.903 PR-AUC=0.904 AUC-ROC=0.885
[VAL-METRICS] VAL (EP 15) | thr*=0.000634 | Prec=0.907 Rec=0.768 Acc=0.844 F1=0.831 AP=0.898 PR-AUC=0.899 AUC-ROC=0.884
[VAL-METRICS] VAL (EP 16) | thr*=0.000001 | Prec=0.518 Rec=0.999 Acc=0.535 F1=0.682 AP=0.755 PR-AUC=0.753 AUC-ROC=0.718
[VAL-METRICS] VAL (EP 17) | thr*=0.000634 | Prec=0.923 Rec=0.708 Acc=0.825 F1=0.802 AP=0.880 PR-AUC=0.885 AUC-ROC=0.839
[VAL-METRICS] VAL (EP 18) | thr*=0.000001 | Prec=0.640 Rec=0.820 Acc=0.679 F1=0.719 AP=0.861 PR-AUC=0.864 AUC-ROC=0.811
[VAL-METRICS] VAL (EP 19) | thr*=0.008229 | Prec=0.846 Rec=0.632 Acc=0.758 F1=0.723 AP=0.821 PR-AUC=0.821 AUC-ROC=0.752
[VAL-METRICS] VAL (EP 20) | thr*=0.000001 | Prec=0.779 Rec=0.787 Acc=0.782 F1=0.783 AP=0.868 PR-AUC=0.881 AUC-ROC=0.830

"""

# === Regex ===
line_re = re.compile(
    r"\bEP\s*(?P<ep>\d+)\b.*?thr\*=(?P<thr>[\d\.eE\-]+)"
    r".*?Prec=(?P<prec>\d+\.\d+)\s+Rec=(?P<rec>\d+\.\d+)\s+Acc=(?P<acc>\d+\.\d+)\s+F1=(?P<f1>\d+\.\d+)"
    r".*?AP=(?P<ap>\d+\.\d+)\s+PR-AUC=(?P<prauc>\d+\.\d+)\s+AUC-ROC=(?P<auc>\d+\.\d+)"
)

# === Parse ===
rows = []
for line in LOGS.splitlines():
    m = line_re.search(line)
    if not m:
        continue
    rows.append({
        "ep": int(m.group("ep")),
        "thr": float(m.group("thr")),
        "prec": float(m.group("prec")),
        "rec": float(m.group("rec")),
        "acc": float(m.group("acc")),
        "f1": float(m.group("f1")),
        "ap": float(m.group("ap")),
        "prauc": float(m.group("prauc")),
        "auc": float(m.group("auc")),
    })

rows.sort(key=lambda d: d["ep"])
episodes = [r["ep"] for r in rows]
get_series = lambda key: np.array([r[key] for r in rows])

metrics = [
    ("prec",  "Precision"),
    ("rec",   "Recall"),
    ("acc",   "Accuracy"),
    ("f1",    "F1 Score"),
    ("ap",    "Average Precision"),
    ("prauc", "PR-AUC"),
    ("auc",   "AUC-ROC"),
    ("thr",   "Threshold (thr*)"),
]

# === Helper for nice Y-axis limits ===
def set_smart_ylim(values):
    vmin, vmax = np.min(values), np.max(values)
    pad = 0.02 * (vmax - vmin)
    plt.ylim(vmin - pad, vmax + pad)
    step = round((vmax - vmin) / 6, 2)
    if step > 0:
        plt.yticks(np.arange(round(vmin, 2), round(vmax + step, 2), step))

# === Generate and save ===
for key, title in metrics:
    vals = get_series(key)
    plt.figure(figsize=(7, 4))
    plt.plot(episodes, vals, marker="o", linewidth=2, color="#007acc")
    plt.xlabel("Episode", fontsize=11)
    plt.ylabel(title, fontsize=11)
    plt.title(f"{title} vs Episode", fontsize=12, fontweight="bold")
    plt.xticks(episodes)
    set_smart_ylim(vals)
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()

    save_path = os.path.join(SAVE_DIR, f"{key}_vs_episode.png")
    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"✅ Saved: {save_path}")

print(f"\nAll plots saved in: {os.path.abspath(SAVE_DIR)}")


✅ Saved: ./val_metric_plots_run12_no_learn/prec_vs_episode.png
✅ Saved: ./val_metric_plots_run12_no_learn/rec_vs_episode.png
✅ Saved: ./val_metric_plots_run12_no_learn/acc_vs_episode.png
✅ Saved: ./val_metric_plots_run12_no_learn/f1_vs_episode.png
✅ Saved: ./val_metric_plots_run12_no_learn/ap_vs_episode.png
✅ Saved: ./val_metric_plots_run12_no_learn/prauc_vs_episode.png
✅ Saved: ./val_metric_plots_run12_no_learn/auc_vs_episode.png
✅ Saved: ./val_metric_plots_run12_no_learn/thr_vs_episode.png

All plots saved in: /content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/val_metric_plots_run12_no_learn
